# Analyse divergence adresse INPI INSEE

Objective(s)

* Création d’une table d’analyse rapprochant la table préparée de l’INPI et la table préparée de l’INSEE
* Analyse des similarités/dissimilarités entre l’adresse de l’INPI et de l’INSEE
* L’analyse comporte le compte du nombre d’observations impactant chacun des cas (1 a 7)
* L’analyse doit comporter pour chacun des cas, l’analyse des divergences en prenant en compte les informations complémentaires de la base de données, a savoir:
  * Information sur établissement
    * Date de création
      * datecreationetablissement 
      * date_début_activité 
    * Etablissement ouvert/fermé
      * etatadministratifetablissement 
      * status_admin 
    * Etablissement Siege 
      * etablissementsiege 
      * status_ets 
  * Information sur l’adresse
    * Code commune
      * codecommuneetablissement 
      * code_commune  
    * Code commune
      * codecommuneetablissement 
      * code_commune  
    * Adresse 
      * Numéro de voie
        * numerovoieetablissement 
        * numero_voie_matching 
      * Type de voie
        * typevoieetablissement 
        * type_voie_matching 
* L’analyse doit aussi comporter le nombre de doublon (les lignes index_id  par cas)
  * Pour un cas donnée
  * Pour un cas et test donnée
* L’analyse doit aussi comporter le nombre de doublon (les sequences sequence_id  par cas)
  * Pour un cas donnée
  * Pour un cas et test donnée  

## Metadata

* Metadata parameters are available here: Ressources_suDYJ#_luZqd
* Task type:
  * Jupyter Notebook
* Users: :
  * Thomas Pernet
* Watchers:
  * Thomas Pernet
* Estimated Log points:
  * One being a simple task, 15 a very difficult one
  *  10
* Task tag
  *  #sql-query,#probability,#matching
* Toggl Tag
  * #datanalaysis
* Instance [AWS/GCP]
  *  
  
## Input Cloud Storage [AWS/GCP]

If link from the internet, save it to the cloud first

### AWS

1. S3
  * File (csv/json) + name and link: 
    * 
    * Notebook construction file (data lineage, md link from Github) 
      * md :
      * py :
2. Athena 
  * Region: eu-west-3 
  * Database: inpi 
    * Table: ets_final_sql  
  * Notebook construction file (data lineage) 
    * md : https://github.com/thomaspernet/InseeInpi_matching/blob/master/Notebooks_matching/Data_preprocessed/programme_matching/01_preparation/03_ETS_add_variables.md
  * Region: Europe (Paris)
  * Database: inpi 
    * Table: insee_final_sql  
  * Notebook construction file (data lineage) 
    * md : 04_ETS_add_variables_insee.md
    
## Destination Output/Delivery

1. Table/Data (AWS/GCP link)
  * Description expected outcome:
    *  La table rassemble l’INSEE et l’INPI et un ensemble de variables connexe pour distinguer les siret
  * AWS
    * Bucket:
      * Link
  * Athena: 
    * Region: Europe (Paris)
    * Database: inpi 
    *  Table:   ets_insee_inpi  
    
## Things to know (Steps, Attention points or new flow of information)

### Sources of information  (meeting notes, Documentation, Query, URL)

1. Other source [Name](link)
  * Source 1: Ensemble fonctions Presto a appliquer sur les arrays. Les fonctions uniques, distinct, intersect sont intéressantes dans notre cas de figure

## Connexion serveur

In [1]:
from awsPy.aws_authorization import aws_connector
from awsPy.aws_athena import service_athena
from awsPy.aws_s3 import service_s3
from pathlib import Path
import pandas as pd
import numpy as np
import os, shutil
bucket = 'calfdata'
path = os.getcwd()
parent_path = str(Path(path).parent)
path_cred = r"{}/credential_AWS.json".format(parent_path)
con = aws_connector.aws_instantiate(credential = path_cred,
                                       region = 'eu-west-3')
client= con.client_boto()
s3 = service_s3.connect_S3(client = client,
                      bucket = 'calfdata', verbose = False) 
athena = service_athena.connect_athena(client = client,
                      bucket = 'calfdata') 

# Functions

La fonction ci dessous va générer le tableau d'analayse via une query, et retourne un dataframe Pandas, tout en stockant le resultat dans le dossier suivant:

- [calfdata/Analyse_cas_similarite_adresse](https://s3.console.aws.amazon.com/s3/buckets/calfdata/Analyse_cas_similarite_adresse/?region=eu-west-3&tab=overview)

In [2]:
a = ["True", "False", "NULL"]
b = range(1,20)


index = pd.MultiIndex.from_product([a, b], names = ["groups", "cnt_test"])

df_ = (pd.DataFrame(index = index)
       .reset_index()
       .sort_values(by = ["cnt_test", "groups"])
       .to_csv('cartesian_table.csv', index = False)
      )

s3.upload_file(file_to_upload = 'cartesian_table.csv',
            destination_in_s3 = 'Temp_table_analyse_similarite')

In [3]:
create_table = """
CREATE EXTERNAL TABLE IF NOT EXISTS inpi.cartesian_table (
`groups`                     string,
`cnt_test`                   int
    )
     ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
    WITH SERDEPROPERTIES (
   'separatorChar' = ',',
   'quoteChar' = '"'
   )
     LOCATION 's3://calfdata/Temp_table_analyse_similarite'
     TBLPROPERTIES ('has_encrypted_data'='false',
              'skip.header.line.count'='1');"""
output = athena.run_query(
        query=create_table,
        database='inpi',
        s3_output='INPI/sql_output'
    )

Execution ID: 389b3e01-f7ed-4139-96ec-eff8ed0c9bdb


## Compte nombre obs par cas

In [4]:
query_count = """
WITH test_proba AS (
  SELECT 
    array_distinct(
      split(adresse_distance_inpi, ' ')
    ) as list_inpi, 
  
    cardinality(
      array_distinct(
        split(adresse_distance_inpi, ' ')
      )
    ) as lenght_list_inpi, 
  
    array_distinct(
      split(adresse_distance_insee, ' ')
    ) as list_insee, 
  
    cardinality(
      array_distinct(
        split(adresse_distance_insee, ' ')
      )
    ) as lenght_list_insee, 
  
    array_distinct(
      array_except(
        split(adresse_distance_insee, ' '), 
        split(adresse_distance_inpi, ' ')
      )
    ) as insee_except, 
  array_distinct(
      array_except(
        split(adresse_distance_inpi, ' '), 
        split(adresse_distance_insee, ' ')
      )
    ) as inpi_except,
    CAST(
      cardinality(
        array_distinct(
          array_intersect(
            split(adresse_distance_inpi, ' '), 
            split(adresse_distance_insee, ' ')
          )
        )
      ) AS DECIMAL(10, 2)
    ) as intersection, 
    CAST(
      cardinality(
        array_distinct(
          array_union(
            split(adresse_distance_inpi, ' '), 
            split(adresse_distance_insee, ' ')
          )
        )
      ) AS DECIMAL(10, 2)
    ) as union_ 
  FROM 
    "inpi"."ets_insee_inpi" -- limit 10
    ) 
SELECT 
 count(*) 
FROM 
  test_proba 
WHERE {}
"""

filter_ = ""

In [5]:
def compte_obs_cas(case = 1):
    """
    """
    
    if case ==1:
        
        filter_= "intersection = union_"
        
    if case ==2:
        
        filter_= "intersection = 0"
    
    if case ==3:
        
        filter_= "lenght_list_inpi = intersection AND intersection != union_"
    
    if case ==4:
        
        filter_= "lenght_list_insee = intersection AND intersection != union_"
    
    if case ==5:
        
        filter_= "cardinality(insee_except) = cardinality(inpi_except) AND intersection != 0 AND cardinality(insee_except) > 0"
    
    if case ==6:
        
        filter_= "cardinality(insee_except) > cardinality(inpi_except) AND intersection != 0 AND cardinality(insee_except) > 0 AND cardinality(inpi_except) > 0"
    
    if case ==7:
        
        filter_= "cardinality(insee_except) < cardinality(inpi_except) AND intersection != 0 AND cardinality(insee_except) > 0 AND cardinality(inpi_except) > 0"
    
    query_ =query_count.format(filter_)
    
    output = athena.run_query(
        query=query_,
        database='inpi',
        s3_output='INPI/sql_output'
    )

    results = False
    
    filename = 'nb_obs_cas_{}.csv'.format(case)
    
    while results != True:
        source_key = "{}/{}.csv".format(
                            'INPI/sql_output',
                            output['QueryExecutionId']
                                   )
        destination_key = "{}/{}".format(
                                'Analyse_cas_similarite_adresse',
                                filename
                            )

        results = s3.copy_object_s3(
                                source_key = source_key,
                                destination_key = destination_key,
                                remove = True
                            )
        
    df_ = (s3.read_df_from_s3(
        key = 'Analyse_cas_similarite_adresse/{}'.format(filename), sep = ',')
          ).values[0][0]
    
    return df_

## Compte nombre duplicate par cas

In [6]:
query_duplicate_cas = """
WITH test_proba AS (
  SELECT 
    {0}, 
    Coalesce(
      try(
        date_parse(
          datecreationetablissement, '%Y-%m-%d'
        )
      ), 
      try(
        date_parse(
          datecreationetablissement, '%Y-%m-%d %hh:%mm:%ss.SSS'
        )
      ), 
      try(
        date_parse(
          datecreationetablissement, '%Y-%m-%d %hh:%mm:%ss'
        )
      ), 
      try(
        cast(
          datecreationetablissement as timestamp
        )
      )
    ) as datecreationetablissement, 
    Coalesce(
      try(
        date_parse(
          "date_début_activité", '%Y-%m-%d'
        )
      ), 
      try(
        date_parse(
          "date_début_activité", '%Y-%m-%d %hh:%mm:%ss.SSS'
        )
      ), 
      try(
        date_parse(
          "date_début_activité", '%Y-%m-%d %hh:%mm:%ss'
        )
      ), 
      try(
        cast(
          "date_début_activité" as timestamp
        )
      )
    ) as date_debut_activite, 
    etatadministratifetablissement, 
    status_admin, 
    etablissementsiege, 
    status_ets, 
    codecommuneetablissement, 
    code_commune, 
    codepostaletablissement, 
    code_postal_matching, 
    numerovoieetablissement, 
    numero_voie_matching, 
    typevoieetablissement, 
    type_voie_matching, 
    adresse_distance_inpi, 
    adresse_distance_insee, 
    array_distinct(
      split(adresse_distance_inpi, ' ')
    ) as list_inpi, 
    cardinality(
      array_distinct(
        split(adresse_distance_inpi, ' ')
      )
    ) as lenght_list_inpi, 
    array_distinct(
      split(adresse_distance_insee, ' ')
    ) as list_insee, 
    cardinality(
      array_distinct(
        split(adresse_distance_insee, ' ')
      )
    ) as lenght_list_insee, 
    array_distinct(
      array_except(
        split(adresse_distance_insee, ' '), 
        split(adresse_distance_inpi, ' ')
      )
    ) as inpi_except, 
    array_distinct(
      array_except(
        split(adresse_distance_inpi, ' '), 
        split(adresse_distance_insee, ' ')
      )
    ) as insee_except, 
    CAST(
      cardinality(
        array_distinct(
          array_intersect(
            split(adresse_distance_inpi, ' '), 
            split(adresse_distance_insee, ' ')
          )
        )
      ) AS DECIMAL(10, 2)
    ) as intersection, 
    CAST(
      cardinality(
        array_distinct(
          array_union(
            split(adresse_distance_inpi, ' '), 
            split(adresse_distance_insee, ' ')
          )
        )
      ) AS DECIMAL(10, 2)
    ) as union_ 
  FROM 
    "inpi"."ets_insee_inpi" -- limit 10
    ) 
SELECT 
  * 
FROM 
  (
    WITH tests AS (
      SELECT 
        {0}, 
        adresse_distance_inpi, 
        adresse_distance_insee, 
        datecreationetablissement, 
        date_debut_activite, 
        CASE WHEN datecreationetablissement = date_debut_activite THEN 'True' WHEN datecreationetablissement IS NULL 
        OR date_debut_activite IS NULL THEN 'NULL' ELSE 'False' END AS test_date, 
        etatadministratifetablissement, 
        status_admin, 
        CASE WHEN etatadministratifetablissement = status_admin THEN 'True' WHEN etatadministratifetablissement = '' 
        OR status_admin = '' THEN 'NULL' ELSE 'False' END AS test_status_admin, 
        etablissementsiege, 
        status_ets, 
        CASE WHEN etablissementsiege = status_ets THEN 'True' WHEN etablissementsiege = '' 
        OR status_ets = '' THEN 'NULL' ELSE 'False' END AS test_siege, 
        codecommuneetablissement, 
        code_commune, 
        CASE WHEN codecommuneetablissement = code_commune THEN 'True' WHEN codecommuneetablissement = '' 
        OR code_commune = '' THEN 'NULL' ELSE 'False' END AS test_code_commune, 
        codepostaletablissement, 
        code_postal_matching, 
        CASE WHEN codepostaletablissement = code_postal_matching THEN 'True' WHEN codepostaletablissement = '' 
        OR code_postal_matching = '' THEN 'NULL' ELSE 'False' END AS test_code_postal, 
        numerovoieetablissement, 
        numero_voie_matching, 
        CASE WHEN numerovoieetablissement = numero_voie_matching THEN 'True' WHEN numerovoieetablissement = '' 
        OR numero_voie_matching = '' THEN 'NULL' ELSE 'False' END AS test_numero_voie, 
        typevoieetablissement, 
        type_voie_matching, 
        CASE WHEN typevoieetablissement = type_voie_matching THEN 'True' WHEN typevoieetablissement = '' 
        OR type_voie_matching = '' THEN 'NULL' ELSE 'False' END AS test_type_voie, 
        list_inpi, 
        list_insee, 
        inpi_except, 
        insee_except, 
        intersection, 
        union_ 
      FROM 
        test_proba 
      WHERE 
        {1}
    ) 
    SELECT 
      count_index_id, 
      COUNT(*) count_duplicate_index_id 
    FROM 
      (
        SELECT 
          {0}, 
          COUNT(*) AS count_index_id 
        FROM 
          tests 
        GROUP BY 
          {0}
      ) 
    GROUP BY 
      count_index_id -- WHERE test_type_voie = 'False'
      ) 
ORDER BY 
  count_index_id,
  count_duplicate_index_id DESC
"""

In [7]:
def compte_dup_cas(var = 'index_id', case = 1):
    """
    """
    
    if case ==1:
        
        filter_= "intersection = union_"
        
    if case ==2:
        
        filter_= "intersection = 0"
    
    if case ==3:
        
        filter_= "lenght_list_inpi = intersection AND intersection != union_"
    
    if case ==4:
        
        filter_= "lenght_list_insee = intersection AND intersection != union_"
    
    if case ==5:
        
        filter_= "cardinality(insee_except) = cardinality(inpi_except) AND intersection != 0 AND cardinality(insee_except) > 0"
    
    if case ==6:
        
        filter_= "cardinality(insee_except) > cardinality(inpi_except) AND intersection != 0 AND cardinality(insee_except) > 0 AND cardinality(inpi_except) > 0"
    
    if case ==7:
        
        filter_= "cardinality(insee_except) < cardinality(inpi_except) AND intersection != 0 AND cardinality(insee_except) > 0 AND cardinality(inpi_except) > 0"
    
    query_ =query_duplicate_cas.format(var, filter_)
    
    output = athena.run_query(
        query=query_,
        database='inpi',
        s3_output='INPI/sql_output'
    )

    results = False
    
    filename = 'nb_dup_cas_{}.csv'.format(case)
    
    while results != True:
        source_key = "{}/{}.csv".format(
                            'INPI/sql_output',
                            output['QueryExecutionId']
                                   )
        destination_key = "{}/{}".format(
                                'Analyse_cas_similarite_adresse',
                                filename
                            )

        results = s3.copy_object_s3(
                                source_key = source_key,
                                destination_key = destination_key,
                                remove = True
                            )
        
    df_ = (s3.read_df_from_s3(
        key = 'Analyse_cas_similarite_adresse/{}'.format(filename), sep = ',')
           .assign(percentage = lambda x: x['count_duplicate_index_id'] / 
                  x['count_duplicate_index_id'].sum())
           .style
           .bar(subset= ['count_duplicate_index_id'],
                   color='#d65f5f')
           .format("{:.2%}", subset =  ['percentage'])
           .format("{:,.0f}", subset =  ['count_duplicate_index_id'])
           
          )
    
    return df_

## Compte nombre obs par cas et test

In [8]:
query_count_case = """
WITH test_proba AS (
  SELECT 
  Coalesce(
         try(date_parse(datecreationetablissement, '%Y-%m-%d')),
         try(date_parse(datecreationetablissement, '%Y-%m-%d %hh:%mm:%ss.SSS')),
         try(date_parse(datecreationetablissement, '%Y-%m-%d %hh:%mm:%ss')),
         try(cast(datecreationetablissement as timestamp))
       )  as datecreationetablissement,

Coalesce(
         try(date_parse("date_début_activité", '%Y-%m-%d')),
         try(date_parse("date_début_activité", '%Y-%m-%d %hh:%mm:%ss.SSS')),
         try(date_parse("date_début_activité", '%Y-%m-%d %hh:%mm:%ss')),
         try(cast("date_début_activité" as timestamp))
  ) as date_debut_activite,
  etatadministratifetablissement, status_admin,
  etablissementsiege,status_ets,
  codecommuneetablissement, code_commune,
  codepostaletablissement, code_postal_matching,
  numerovoieetablissement, numero_voie_matching,
  typevoieetablissement, type_voie_matching,
  
    array_distinct(
      split(adresse_distance_inpi, ' ')
    ) as list_inpi, 
  
    cardinality(
      array_distinct(
        split(adresse_distance_inpi, ' ')
      )
    ) as lenght_list_inpi, 
  
    array_distinct(
      split(adresse_distance_insee, ' ')
    ) as list_insee, 
  
    cardinality(
      array_distinct(
        split(adresse_distance_insee, ' ')
      )
    ) as lenght_list_insee,
  
  array_distinct(
              array_except(
                split(adresse_distance_insee, ' '), 
                split(adresse_distance_inpi, ' ')
              )
            )as inpi_except, 
  array_distinct(
              array_except(
                split(adresse_distance_inpi, ' '), 
                split(adresse_distance_insee, ' ')
              )
            )as insee_except,
  
    CAST(
      cardinality(
        array_distinct(
          array_intersect(
            split(adresse_distance_inpi, ' '), 
            split(adresse_distance_insee, ' ')
          )
        )
      ) AS DECIMAL(10, 2)
    ) as intersection, 
    CAST(
      cardinality(
        array_distinct(
          array_union(
            split(adresse_distance_inpi, ' '), 
            split(adresse_distance_insee, ' ')
          )
        )
      ) AS DECIMAL(10, 2)
    ) as union_
  FROM "inpi"."ets_insee_inpi"-- limit 10
  )
  SELECT *
  FROM (WITH tests AS (
    SELECT 
  datecreationetablissement,date_debut_activite,
  CASE WHEN datecreationetablissement = date_debut_activite THEN 'True'
  WHEN datecreationetablissement IS NULL OR date_debut_activite IS NULL THEN 'NULL'
  ELSE 'False' END AS test_date,
  
  etatadministratifetablissement,status_admin,
  CASE WHEN etatadministratifetablissement = status_admin THEN 'True' 
  WHEN etatadministratifetablissement = '' OR status_admin = '' THEN 'NULL'
  ELSE 'False' END AS test_status_admin,
  
  etablissementsiege,status_ets,
  CASE WHEN etablissementsiege = status_ets THEN 'True' 
  WHEN etablissementsiege = '' OR status_ets = '' THEN 'NULL'
  ELSE 'False' END AS test_siege,
  
  codecommuneetablissement,code_commune,
  CASE WHEN codecommuneetablissement = code_commune THEN 'True' 
  WHEN codecommuneetablissement = '' OR code_commune = '' THEN 'NULL'
  ELSE 'False' END AS test_code_commune,
  
  codepostaletablissement,code_postal_matching,
  CASE WHEN codepostaletablissement = code_postal_matching THEN 'True' 
  WHEN codepostaletablissement = '' OR code_postal_matching = '' THEN 'NULL'
  ELSE 'False' END AS test_code_postal,
  
  numerovoieetablissement,numero_voie_matching,
  CASE WHEN numerovoieetablissement = numero_voie_matching THEN 'True' 
  WHEN numerovoieetablissement = '' OR numero_voie_matching = '' THEN 'NULL'
  ELSE 'False' END AS test_numero_voie,
  
  typevoieetablissement,type_voie_matching,
  CASE WHEN typevoieetablissement = type_voie_matching THEN 'True'
  WHEN typevoieetablissement = '' OR type_voie_matching = '' THEN 'NULL'
  ELSE 'False' END AS test_type_voie,
  
  list_inpi, list_insee, inpi_except, insee_except, intersection, union_
  
  FROM test_proba
  WHERE {}
    )
        
  SELECT DISTINCT(test_date) as groups,
        count_test_date,
        count_test_status_admin,
        count_test_siege,
        count_test_commune,
        count_test_cp,
        count_test_num_voie,
        count_test_type_voie
        
  FROM tests
  LEFT JOIN (
    SELECT test_date as groups,  count(*) as count_test_date
    FROM tests
    GROUP BY test_date
    ) as date_
  ON date_.groups = tests.test_date
        
  LEFT JOIN (
    SELECT test_status_admin as groups,  count(*) as count_test_status_admin
    FROM tests
    GROUP BY test_status_admin
    ) as admin_
  ON admin_.groups = tests.test_date      
        
LEFT JOIN (
    SELECT test_siege as groups,  count(*) as count_test_siege
    FROM tests
    GROUP BY test_siege
    ) as siege_
  ON siege_.groups = tests.test_date

LEFT JOIN (
      SELECT test_code_commune as groups,  count(*) as count_test_commune
      FROM tests
      GROUP BY test_code_commune
      ) as code_commune_
    ON code_commune_.groups = tests.test_date

LEFT JOIN (
        SELECT test_code_postal as groups,  count(*) as count_test_cp
        FROM tests
        GROUP BY test_code_postal
        ) as cp_
      ON cp_.groups = tests.test_date

LEFT JOIN (
          SELECT test_numero_voie as groups,  count(*) as count_test_num_voie
          FROM tests
          GROUP BY test_numero_voie
          ) as num_voie_
        ON num_voie_.groups = tests.test_date

LEFT JOIN (
            SELECT test_type_voie as groups,  count(*) as count_test_type_voie
            FROM tests
            GROUP BY test_type_voie
            ) as type_voie_
          ON type_voie_.groups = tests.test_date
 )
"""

In [9]:
def generate_analytical_table(case = 1):
    """
    """
    if case ==1:
        
        filter_= "intersection = union_"
        
    if case ==2:
        
        filter_= "intersection = 0"
    
    if case ==3:
        
        filter_= "lenght_list_inpi = intersection AND intersection != union_"
    
    if case ==4:
        
        filter_= "lenght_list_insee = intersection AND intersection != union_"
    
    if case ==5:
        
        filter_= "cardinality(insee_except) = cardinality(inpi_except) AND intersection != 0 AND cardinality(insee_except) > 0"
    
    if case ==6:
        
        filter_= "cardinality(insee_except) > cardinality(inpi_except) AND intersection != 0 AND cardinality(insee_except) > 0 AND cardinality(inpi_except) > 0"
    
    if case ==7:
        
        filter_= "cardinality(insee_except) < cardinality(inpi_except) AND intersection != 0 AND cardinality(insee_except) > 0 AND cardinality(inpi_except) > 0"

    output = athena.run_query(
        query=query_count_case.format(filter_),
        database='inpi',
        s3_output='INPI/sql_output'
    )

    results = False
    filename = 'cas_{}.csv'.format(case)
    
    while results != True:
        source_key = "{}/{}.csv".format(
                            'INPI/sql_output',
                            output['QueryExecutionId']
                                   )
        destination_key = "{}/{}".format(
                                'Analyse_cas_similarite_adresse',
                                filename
                            )

        results = s3.copy_object_s3(
                                source_key = source_key,
                                destination_key = destination_key,
                                remove = True
                            )

    test_1 = (s3.read_df_from_s3(
        key = 'Analyse_cas_similarite_adresse/{}'.format(filename), sep = ',')
              .assign(test = 'cas_{}'.format(case),
                     count_test_date_pct = lambda x: x['count_test_date'] / x['count_test_date'].sum(),
                     count_test_status_admin_pct = lambda x: x['count_test_status_admin'] / x['count_test_status_admin'].sum(),
                     count_test_siege_pct = lambda x: x['count_test_siege'] / x['count_test_siege'].sum(),
                     count_test_commune_pct = lambda x: x['count_test_commune'] / x['count_test_commune'].sum(),
                     count_test_cp_pct = lambda x: x['count_test_cp'] / x['count_test_cp'].sum(),
                     count_test_num_voie_pct = lambda x: x['count_test_num_voie'] / x['count_test_num_voie'].sum(),
                     count_test_type_voie_pct = lambda x: x['count_test_type_voie'] / x['count_test_type_voie'].sum(),
                     )

              .replace({'groups' :{np.nan: 'Null'}})
              #.set_index(['test'])
              .reindex(columns = [
                  'test',
                  'groups',
                   'count_test_num_voie','count_test_num_voie_pct',
                  'count_test_type_voie','count_test_type_voie_pct',
                  'count_test_commune','count_test_commune_pct',
                  'count_test_date','count_test_date_pct',
                  'count_test_status_admin','count_test_status_admin_pct',
                  'count_test_siege','count_test_siege_pct',
                  
                  'count_test_cp','count_test_cp_pct',
                 
              ])
              .fillna(0)
              .style
              .format("{:,.0f}", subset =  ['count_test_date',
                                            'count_test_status_admin',
                                            'count_test_siege',
                                            'count_test_commune',
                                            'count_test_cp',
                                            'count_test_num_voie',
                                            'count_test_type_voie'])
              .format("{:.2%}", subset =  ['count_test_date_pct',
                                           'count_test_status_admin_pct',
                                           'count_test_siege_pct',
                                           'count_test_commune_pct',
                                           'count_test_cp_pct',
                                           'count_test_num_voie_pct',
                                           'count_test_type_voie_pct'])
              .bar(subset= ['count_test_date',
                                            'count_test_status_admin',
                                            'count_test_siege',
                                            'count_test_commune',
                                            'count_test_cp',
                                            'count_test_num_voie',
                                            'count_test_type_voie'],
                   color='#d65f5f')
              #.unstack(0)
             )    
    
    return test_1

## Compte nombre duplicate par cas et test

In [10]:
query_dup_cas = """
WITH test_proba AS (
  SELECT 
  {0},
  Coalesce(
         try(date_parse(datecreationetablissement, '%Y-%m-%d')),
         try(date_parse(datecreationetablissement, '%Y-%m-%d %hh:%mm:%ss.SSS')),
         try(date_parse(datecreationetablissement, '%Y-%m-%d %hh:%mm:%ss')),
         try(cast(datecreationetablissement as timestamp))
       )  as datecreationetablissement,

Coalesce(
         try(date_parse("date_début_activité", '%Y-%m-%d')),
         try(date_parse("date_début_activité", '%Y-%m-%d %hh:%mm:%ss.SSS')),
         try(date_parse("date_début_activité", '%Y-%m-%d %hh:%mm:%ss')),
         try(cast("date_début_activité" as timestamp))
  ) as date_debut_activite,
  etatadministratifetablissement, status_admin,
  etablissementsiege,status_ets,
  codecommuneetablissement, code_commune,
  codepostaletablissement, code_postal_matching,
  numerovoieetablissement, numero_voie_matching,
  typevoieetablissement, type_voie_matching,
  
    array_distinct(
      split(adresse_distance_inpi, ' ')
    ) as list_inpi, 
  
    cardinality(
      array_distinct(
        split(adresse_distance_inpi, ' ')
      )
    ) as lenght_list_inpi, 
  
    array_distinct(
      split(adresse_distance_insee, ' ')
    ) as list_insee, 
  
    cardinality(
      array_distinct(
        split(adresse_distance_insee, ' ')
      )
    ) as lenght_list_insee,
  
  array_distinct(
              array_except(
                split(adresse_distance_insee, ' '), 
                split(adresse_distance_inpi, ' ')
              )
            )as inpi_except, 
  array_distinct(
              array_except(
                split(adresse_distance_inpi, ' '), 
                split(adresse_distance_insee, ' ')
              )
            )as insee_except,
  
    CAST(
      cardinality(
        array_distinct(
          array_intersect(
            split(adresse_distance_inpi, ' '), 
            split(adresse_distance_insee, ' ')
          )
        )
      ) AS DECIMAL(10, 2)
    ) as intersection, 
    CAST(
      cardinality(
        array_distinct(
          array_union(
            split(adresse_distance_inpi, ' '), 
            split(adresse_distance_insee, ' ')
          )
        )
      ) AS DECIMAL(10, 2)
    ) as union_
  FROM "inpi"."ets_insee_inpi"-- limit 10
  )
  SELECT *
  FROM (WITH tests AS (
    SELECT 
    {0},
  datecreationetablissement,date_debut_activite,
  CASE WHEN datecreationetablissement = date_debut_activite THEN 'True'
  WHEN datecreationetablissement IS NULL OR date_debut_activite IS NULL THEN 'NULL'
  ELSE 'False' END AS test_date,
  
  etatadministratifetablissement,status_admin,
  CASE WHEN etatadministratifetablissement = status_admin THEN 'True' 
  WHEN etatadministratifetablissement = '' OR status_admin = '' THEN 'NULL'
  ELSE 'False' END AS test_status_admin,
  
  etablissementsiege,status_ets,
  CASE WHEN etablissementsiege = status_ets THEN 'True' 
  WHEN etablissementsiege = '' OR status_ets = '' THEN 'NULL'
  ELSE 'False' END AS test_siege,
  
  codecommuneetablissement,code_commune,
  CASE WHEN codecommuneetablissement = code_commune THEN 'True' 
  WHEN codecommuneetablissement = '' OR code_commune = '' THEN 'NULL'
  ELSE 'False' END AS test_code_commune,
  
  codepostaletablissement,code_postal_matching,
  CASE WHEN codepostaletablissement = code_postal_matching THEN 'True' 
  WHEN codepostaletablissement = '' OR code_postal_matching = '' THEN 'NULL'
  ELSE 'False' END AS test_code_postal,
  
  numerovoieetablissement,numero_voie_matching,
  CASE WHEN numerovoieetablissement = numero_voie_matching THEN 'True' 
  WHEN numerovoieetablissement = '' OR numero_voie_matching = '' THEN 'NULL'
  ELSE 'False' END AS test_numero_voie,
  
  typevoieetablissement,type_voie_matching,
  CASE WHEN typevoieetablissement = type_voie_matching THEN 'True'
  WHEN typevoieetablissement = '' OR type_voie_matching = '' THEN 'NULL'
  ELSE 'False' END AS test_type_voie,
  
  list_inpi, list_insee, inpi_except, insee_except, intersection, union_
  
  FROM test_proba
  WHERE {1}
    )
        SELECT 
        cartesian_table.groups, 
        cnt_test,
        cnt_index_date,
        cnt_index_admin,
        cnt_index_siege,
        cnt_index_commune,
        cnt_index_cp,
        cnt_index_num_voie,
        cnt_index_type_voie

        
  FROM cartesian_table
        
  LEFT JOIN (
    
    SELECT groups, cnt_test_date, COUNT(*) AS cnt_index_date   
    FROM (    
    SELECT test_date as groups,{0},   count(*) as cnt_test_date
    FROM tests
    GROUP BY test_date, {0}
    ) as date_
    GROUP BY groups, cnt_test_date
        ) as count_dup_date
   ON count_dup_date.groups = cartesian_table.groups and
        count_dup_date.cnt_test_date = cartesian_table.cnt_test
        
   LEFT JOIN (
    
    SELECT groups, cnt_test_admin, COUNT(*) AS cnt_index_admin
    FROM (    
    SELECT test_status_admin as groups,{0}, count(*) as cnt_test_admin
    FROM tests
    GROUP BY test_status_admin, {0}
    ) as admin_
    GROUP BY groups, cnt_test_admin
        ) as count_dup_admin_
   ON count_dup_admin_.groups = cartesian_table.groups and
        count_dup_admin_.cnt_test_admin = cartesian_table.cnt_test   
        
   
   LEFT JOIN (
    
    SELECT groups, cnt_test_siege, COUNT(*) AS cnt_index_siege
    FROM (    
    SELECT test_siege as groups,{0}, count(*) as cnt_test_siege
    FROM tests
    GROUP BY test_siege, {0}
    ) as admin_
    GROUP BY groups, cnt_test_siege
        ) as count_dup_siege_
   ON count_dup_siege_.groups = cartesian_table.groups and
        count_dup_siege_.cnt_test_siege = cartesian_table.cnt_test
        
   LEFT JOIN (
    
    SELECT groups, cnt_test_commune, COUNT(*) AS cnt_index_commune
    FROM (    
    SELECT test_code_commune as groups,{0}, count(*) as cnt_test_commune
    FROM tests
    GROUP BY test_code_commune, {0}
    ) as siege_
    GROUP BY groups, cnt_test_commune
        ) as count_dup_commune_
   ON count_dup_commune_.groups = cartesian_table.groups and
        count_dup_commune_.cnt_test_commune = cartesian_table.cnt_test
        
    LEFT JOIN (
    
    SELECT groups, cnt_test_cp, COUNT(*) AS cnt_index_cp
    FROM (    
    SELECT test_code_postal as groups,{0}, count(*) as cnt_test_cp
    FROM tests
    GROUP BY test_code_postal, {0}
    ) as cp_
    GROUP BY groups, cnt_test_cp
        ) as count_dup_cp_
   ON count_dup_cp_.groups = cartesian_table.groups and
        count_dup_cp_.cnt_test_cp = cartesian_table.cnt_test
        
   LEFT JOIN (
    
    SELECT groups, cnt_test_num_voie, COUNT(*) AS cnt_index_num_voie
    FROM (    
    SELECT test_numero_voie as groups,{0}, count(*) as cnt_test_num_voie
    FROM tests
    GROUP BY test_numero_voie, {0}
    ) as num_voie_
    GROUP BY groups, cnt_test_num_voie
        ) as count_dup_num_voie_
   ON count_dup_num_voie_.groups = cartesian_table.groups and
        count_dup_num_voie_.cnt_test_num_voie = cartesian_table.cnt_test
        
   LEFT JOIN (
    
    SELECT groups, cnt_test_type_voie, COUNT(*) AS cnt_index_type_voie
    FROM (    
    SELECT test_type_voie as groups,{0}, count(*) as cnt_test_type_voie
    FROM tests
    GROUP BY test_type_voie, {0}
    ) as type_voie_
    GROUP BY groups, cnt_test_type_voie
        ) as count_dup_type_voie_
   ON count_dup_type_voie_.groups = cartesian_table.groups and
        count_dup_type_voie_.cnt_test_type_voie = cartesian_table.cnt_test
   
        )
   ORDER BY cnt_test ASC, groups
"""

In [11]:
def generate_analytical_table_dup(var = 'index_id', case = 1):
    """
    """
    if case ==1:
        
        filter_= "intersection = union_"
        
    if case ==2:
        
        filter_= "intersection = 0"
    
    if case ==3:
        
        filter_= "lenght_list_inpi = intersection AND intersection != union_"
    
    if case ==4:
        
        filter_= "lenght_list_insee = intersection AND intersection != union_"
    
    if case ==5:
        
        filter_= "cardinality(insee_except) = cardinality(inpi_except) AND intersection != 0 AND cardinality(insee_except) > 0"
    
    if case ==6:
        
        filter_= "cardinality(insee_except) > cardinality(inpi_except) AND intersection != 0 AND cardinality(insee_except) > 0 AND cardinality(inpi_except) > 0"
    
    if case ==7:
        
        filter_= "cardinality(insee_except) < cardinality(inpi_except) AND intersection != 0 AND cardinality(insee_except) > 0 AND cardinality(inpi_except) > 0"

    output = athena.run_query(
        query=query_dup_cas.format(var, filter_),
        database='inpi',
        s3_output='INPI/sql_output'
    )

    results = False
    filename = 'cas_dup_{}.csv'.format(case)
    
    while results != True:
        source_key = "{}/{}.csv".format(
                            'INPI/sql_output',
                            output['QueryExecutionId']
                                   )
        destination_key = "{}/{}".format(
                                'Analyse_cas_similarite_adresse',
                                filename
                            )

        results = s3.copy_object_s3(
                                source_key = source_key,
                                destination_key = destination_key,
                                remove = True
                            )

    test_1 = (s3.read_df_from_s3(
        key = 'Analyse_cas_similarite_adresse/{}'.format(filename), sep = ',')
              .assign(test = 'cas_{}'.format(case),
                     count_test_date_pct = lambda x: x['cnt_index_date'] / x['cnt_index_date'].sum(),
                     count_test_status_admin_pct = lambda x: x['cnt_index_admin'] / x['cnt_index_admin'].sum(),
                     count_test_siege_pct = lambda x: x['cnt_index_siege'] / x['cnt_index_siege'].sum(),
                     count_test_commune_pct = lambda x: x['cnt_index_commune'] / x['cnt_index_commune'].sum(),
                     count_test_cp_pct = lambda x: x['cnt_index_cp'] / x['cnt_index_cp'].sum(),
                     count_test_num_voie_pct = lambda x: x['cnt_index_num_voie'] / x['cnt_index_num_voie'].sum(),
                     count_test_type_voie_pct = lambda x: x['cnt_index_type_voie'] / x['cnt_index_type_voie'].sum(),
                     )

              .replace({'groups' :{np.nan: 'Null'}})
              #.set_index(['test'])
              .reindex(columns = [
                  'test',
                  'groups',
                  "cnt_test",
                  'cnt_index_num_voie','count_test_num_voie_pct',
                  'cnt_index_type_voie','count_test_type_voie_pct',
                  'cnt_index_commune','count_test_commune_pct',
                  'cnt_index_date','count_test_date_pct',
                  'cnt_index_admin','count_test_status_admin_pct',
                  'cnt_index_siege','count_test_siege_pct',        
                  'cnt_index_cp','count_test_cp_pct',
                  
                  
              ])
              .fillna(0)
              .style
              .format("{:,.0f}", subset =  ['cnt_index_date',
                                            'cnt_index_admin',
                                            'cnt_index_siege',
                                            'cnt_index_commune',
                                            'cnt_index_cp',
                                            'cnt_index_num_voie',
                                            'cnt_index_type_voie'])
              .format("{:.2%}", subset =  ['count_test_date_pct',
                                           'count_test_status_admin_pct',
                                           'count_test_siege_pct',
                                           'count_test_commune_pct',
                                           'count_test_cp_pct',
                                           'count_test_num_voie_pct',
                                           'count_test_type_voie_pct'])
              .bar(subset= ['cnt_index_date',
                                            'cnt_index_admin',
                                            'cnt_index_siege',
                                            'cnt_index_commune',
                                            'cnt_index_cp',
                                            'cnt_index_num_voie',
                                            'cnt_index_type_voie'],
                   color='#d65f5f')
              #.unstack(0)
             )    
    
    return test_1

# Creation table analyse

## Full Pipeline

*   Dans ce notebook, tous les codes SQL pour faire la siretisation seront présent de manière atomique afin de faciliter l’écriture des US. 
   * La première query consiste à rapprocher les deux tables INPI & INSEE
   * La second partie consiste a calculer Levenshtein edit distance sur l’adresse et l’enseigne 
   * La troisième partie consiste a calculer la distance de Jaccard sur l’adresse (au niveau de la lettre) et l’enseigne
   * La quatrième partie consiste a calculer la présence d’un des mots de l’adresse de l’INPI dans l’adresse de l’INSEE  
   * La cinquième partie consiste a calculer la distance de Jaccard sur l’adresse au niveau du mot 

In [12]:
query = """
CREATE TABLE inpi.ets_insee_inpi WITH (format = 'PARQUET') AS WITH insee_inpi AS (
  SELECT 
    index_id, 
    sequence_id, 
    count_initial_insee, 
    ets_final_sql.siren, 
    siret, 
    code_greffe, 
    nom_greffe, 
    numero_gestion, 
    id_etablissement, 
    status, 
    origin, 
    date_greffe, 
    file_timestamp, 
    datecreationetablissement, 
    "date_début_activité", 
    libelle_evt, 
    last_libele_evt, 
    etatadministratifetablissement, 
    status_admin, 
    type, 
    etablissementsiege, 
    status_ets, 
    adresse_reconstituee_inpi, 
    adresse_reconstituee_insee, 
    adresse_regex_inpi, 
    adresse_distance_inpi, 
    adresse_distance_insee, 
    list_numero_voie_matching_inpi, 
    list_numero_voie_matching_insee, 
    numerovoieetablissement, 
    numero_voie_matching, 
    typevoieetablissement, 
    type_voie_matching, 
    ets_final_sql.code_postal_matching, 
    ets_final_sql.ville_matching, 
    codecommuneetablissement, 
    code_commune, 
    enseigne, 
    enseigne1etablissement, 
    enseigne2etablissement, 
    enseigne3etablissement 
  FROM 
    ets_final_sql 
    INNER JOIN (
      SELECT 
        count_initial_insee, 
        siren, 
        siret, 
        datecreationetablissement, 
        etablissementsiege, 
        etatadministratifetablissement, 
        codepostaletablissement, 
        codecommuneetablissement, 
        ville_matching, 
        list_numero_voie_matching_insee, 
        numerovoieetablissement, 
        typevoieetablissement, 
        adresse_reconstituee_insee, 
        adresse_distance_insee, 
        enseigne1etablissement, 
        enseigne2etablissement, 
        enseigne3etablissement 
      FROM 
        insee_final_sql
    ) as insee ON ets_final_sql.siren = insee.siren 
    AND ets_final_sql.ville_matching = insee.ville_matching 
    AND ets_final_sql.code_postal_matching = insee.codepostaletablissement 
  WHERE 
    status != 'IGNORE'
) 
SELECT 
  index_id, 
  sequence_id, 
  count_initial_insee, 
  siren, 
  siret, 
  code_greffe, 
  nom_greffe, 
  numero_gestion, 
  id_etablissement, 
  status, 
  origin, 
  date_greffe, 
  file_timestamp, 
  datecreationetablissement, 
  "date_début_activité", 
  libelle_evt, 
  last_libele_evt, 
  etatadministratifetablissement, 
  status_admin, 
  type, 
  etablissementsiege, 
  status_ets, 
  adresse_reconstituee_inpi, 
  adresse_reconstituee_insee, 
  adresse_regex_inpi, 
  adresse_distance_inpi, 
  adresse_distance_insee, 
  (
    CAST(
      cardinality(
        array_distinct(
          split(adresse_distance_inpi, ' ')
        )
      ) AS DECIMAL(10, 2)
    ) / (
      CAST(
        cardinality(
          array_distinct(
            split(adresse_distance_insee, ' ')
          )
        ) AS DECIMAL(10, 2)
      )
    )
  ) / NULLIF(
    CAST(
      cardinality(
        array_distinct(
          array_except(
            split(adresse_distance_insee, ' '), 
            split(adresse_distance_inpi, ' ')
          )
        )
      ) AS DECIMAL(10, 2)
    ), 
    0
  )* (
    cardinality(
      array_distinct(
        split(adresse_distance_inpi, ' ')
      )
    )* cardinality(
      array_distinct(
        split(adresse_distance_insee, ' ')
      )
    )
  )/(
    NULLIF(
      CAST(
        cardinality(
          array_distinct(
            array_union(
              split(adresse_distance_inpi, ' '), 
              split(adresse_distance_insee, ' ')
            )
          )
        ) AS DECIMAL(10, 2)
      ), 
      0
    ) * NULLIF(
      CAST(
        cardinality(
          array_distinct(
            array_intersect(
              split(adresse_distance_inpi, ' '), 
              split(adresse_distance_insee, ' ')
            )
          )
        ) AS DECIMAL(10, 2)
      ), 
      0
    )
  ) as score_pairing, 
  CASE WHEN cardinality(
    array_distinct(
      split(adresse_distance_inpi, ' ')
    )
  ) = 0 THEN NULL ELSE array_distinct(
    split(adresse_distance_inpi, ' ')
  ) END as liste_distinct_inpi, 
  CASE WHEN cardinality(
    array_distinct(
      split(adresse_distance_insee, ' ')
    )
  ) = 0 THEN NULL ELSE array_distinct(
    split(adresse_distance_insee, ' ')
  ) END as liste_distinct_insee, 
  CASE WHEN cardinality(
    array_distinct(
      array_except(
        split(adresse_distance_insee, ' '), 
        split(adresse_distance_inpi, ' ')
      )
    )
  ) = 0 THEN NULL ELSE array_distinct(
    array_except(
      split(adresse_distance_insee, ' '), 
      split(adresse_distance_inpi, ' ')
    )
  ) END as insee_exclusion, 
  CASE WHEN cardinality(
    array_distinct(
      array_except(
        split(adresse_distance_inpi, ' '), 
        split(adresse_distance_insee, ' ')
      )
    )
  ) = 0 THEN NULL ELSE array_distinct(
    array_except(
      split(adresse_distance_inpi, ' '), 
      split(adresse_distance_insee, ' ')
    )
  ) END as inpi_exclusion, 
  regexp_like(
    adresse_reconstituee_insee, adresse_regex_inpi
  ) as regex_adresse, 
  list_numero_voie_matching_inpi, 
  list_numero_voie_matching_insee, 
  numerovoieetablissement, 
  numero_voie_matching, 
  typevoieetablissement, 
  type_voie_matching, 
  code_postal_matching, 
  ville_matching, 
  codecommuneetablissement, 
  code_commune, 
  enseigne, 
  enseigne1etablissement, 
  enseigne2etablissement, 
  enseigne3etablissement, 
  levenshtein_distance(
    enseigne, enseigne1etablissement
  ) as edit_enseigne1, 
  levenshtein_distance(
    enseigne, enseigne2etablissement
  ) as edit_enseigne2, 
  levenshtein_distance(
    enseigne, enseigne3etablissement
  ) as edit_enseigne3, 
  1 - CAST(
    cardinality(
      array_intersect(
        regexp_extract_all(enseigne, '(\d+)|([A-Z])'), 
        regexp_extract_all(
          enseigne1etablissement, '(\d+)|([A-Z])'
        )
      )
    ) AS DECIMAL(10, 2)
  ) / NULLIF(
    CAST(
      cardinality(
        array_union(
          regexp_extract_all(enseigne, '(\d+)|([A-Z])'), 
          regexp_extract_all(
            enseigne1etablissement, '(\d+)|([A-Z])'
          )
        )
      ) AS DECIMAL(10, 2)
    ), 
    0
  ) as jaccard_enseigne1_lettre, 
  1 - CAST(
    cardinality(
      array_intersect(
        regexp_extract_all(enseigne, '(\d+)|([A-Z])'), 
        regexp_extract_all(
          enseigne2etablissement, '(\d+)|([A-Z])'
        )
      )
    ) AS DECIMAL(10, 2)
  ) / NULLIF(
    CAST(
      cardinality(
        array_union(
          regexp_extract_all(enseigne, '(\d+)|([A-Z])'), 
          regexp_extract_all(
            enseigne2etablissement, '(\d+)|([A-Z])'
          )
        )
      ) AS DECIMAL(10, 2)
    ), 
    0
  ) as jaccard_enseigne2_lettre, 
  1 - CAST(
    cardinality(
      array_intersect(
        regexp_extract_all(enseigne, '(\d+)|([A-Z])'), 
        regexp_extract_all(
          enseigne3etablissement, '(\d+)|([A-Z])'
        )
      )
    ) AS DECIMAL(10, 2)
  ) / NULLIF(
    CAST(
      cardinality(
        array_union(
          regexp_extract_all(enseigne, '(\d+)|([A-Z])'), 
          regexp_extract_all(
            enseigne3etablissement, '(\d+)|([A-Z])'
          )
        )
      ) AS DECIMAL(10, 2)
    ), 
    0
  ) as jaccard_enseigne3_lettre 
FROM 
  insee_inpi

"""

# Evaluation nombre de cas

## Similarité entre deux adresses

Le rapprochement entre les deux tables, à savoir l’INSEE et l’INPI, va amener à la création de deux vecteurs d’adresse. Un vecteur avec des mots contenus spécifiquement à l’INSEE, et un second vecteur avec les mots de l’adresse de l’INPI. Notre objectif est de comparé ses deux vecteurs pour définir si ils sont identiques ou non. Nous avons distingué 7 cas de figures possibles entre les deux vecteurs (figure 1).

![](https://drive.google.com/uc?export=view&id=1Qj_HooHrhFYSuTsoqFbl4Vxy9tN3V5Bu)

## Définition

![](https://upload.wikimedia.org/wikipedia/commons/thumb/1/1f/Intersection_of_sets_A_and_B.svg/400px-Intersection_of_sets_A_and_B.svg.png)



La table `ets_insee_inpi` contient 11 600 551 observations

In [13]:
initial_obs = 11600551

## Tableau recapitulatif

|   Cas de figure | Titre                   |   Total |   Total cumulé |   pourcentage |   Pourcentage cumulé | Comment                 |
|----------------:|:------------------------|--------:|---------------:|--------------:|---------------------:|:------------------------|
|               1 | similarité parfaite     | 7775392 |        7775392 |     0.670261  |             0.670261 | Match parfait           |
|               2 | Exclusion parfaite      |  974444 |        8749836 |     0.0839998 |             0.75426  | Exclusion parfaite      |
|               3 | Match partiel parfait   |  407404 |        9157240 |     0.0351194 |             0.78938  | Match partiel parfait   |
|               4 | Match partiel parfait   |  558992 |        9716232 |     0.0481867 |             0.837566 | Match partiel parfait   |
|               5 | Match partiel compliqué | 1056406 |       10772638 |     0.0910652 |             0.928632 | Match partiel compliqué |
|               6 | Match partiel compliqué |  361242 |       11133880 |     0.0311401 |             0.959772 | Match partiel compliqué |
|               7 | Match partiel compliqué |  466671 |       11600551 |     0.0402283 |             1        | Match partiel compliqué |

In [14]:
dic_ = {
    'Cas de figure': [], 
    'Titre': [], 
    'Total': [], 
    'Total cumulé': [], 
    'pourcentage': [], 
    'Pourcentage cumulé': [], 
    'Comment': [], 
}

## Cas de figure 1: similarité parfaite

* Definition: Les mots dans l’adresse de l’INPI sont égales aux mots dans l’adresse de l’INSEE
- Math definition: $\frac{|INSEE \cap INPI|}{|INSEE|+|INPI|-|INSEE \cap INPI|} =1$
- Règle: $ \text{intersection} = \text{union} \rightarrow \text{cas 1}$
* Query [case 1](https://eu-west-3.console.aws.amazon.com/athena/home?region=eu-west-3#query/history/24e58c22-4a67-4a9e-b98d-4eb9d65e7f27)

| list_inpi              | list_insee             | insee_except | intersection | union_ |
|------------------------|------------------------|--------------|--------------|--------|
| [BOULEVARD, HAUSSMANN] | [BOULEVARD, HAUSSMANN] | []           | 2            | 2      |
| [QUAI, GABUT]          | [QUAI, GABUT]          | []           | 2            | 2      |
| [BOULEVARD, VOLTAIRE]  | [BOULEVARD, VOLTAIRE]  | []           | 2            | 2      |

- Nombre d'observation:  7 775 392 
    - Percentage initial: 0.67

In [15]:
cas_1 =  compte_obs_cas(case= 1)

Execution ID: c88a483a-2b8d-4d3e-a29b-4131cc7847ad


In [16]:
dic_['Cas de figure'].append(1)
dic_['Titre'].append('similarité parfaite')
dic_['Total'].append(cas_1)
dic_['Total cumulé'].append(cas_1)
dic_['pourcentage'].append(cas_1/initial_obs)
dic_['Pourcentage cumulé'].append(cas_1/initial_obs)
dic_['Comment'].append("Match parfait")

In [17]:
generate_analytical_table(case = 1)

Execution ID: 6b7731a4-b823-4993-83dd-9f2305a15f17


,test,groups,count_test_num_voie,count_test_num_voie_pct,count_test_type_voie,count_test_type_voie_pct,count_test_commune,count_test_commune_pct,count_test_date,count_test_date_pct,count_test_status_admin,count_test_status_admin_pct,count_test_siege,count_test_siege_pct,count_test_cp,count_test_cp_pct
0,cas_1,Null,"1,084,365",13.95%,"782,532",10.06%,"115,742",1.49%,"2,452,952",31.55%,0,0.00%,0,0.00%,0,0.00%
1,cas_1,False,"190,078",2.44%,"17,979",0.23%,"3,162",0.04%,"1,506,160",19.37%,"1,381,738",17.77%,"3,164,336",40.70%,0,0.00%
2,cas_1,True,"6,500,949",83.61%,"6,974,881",89.70%,"7,656,488",98.47%,"3,816,280",49.08%,"6,393,654",82.23%,"4,611,056",59.30%,"7,775,392",100.00%


Analyse Index

In [18]:
compte_dup_cas(var = 'index_id', case = 1)

Execution ID: 4ce813bb-76d5-4265-96dd-f184b83e9b88


,count_index_id,count_duplicate_index_id,percentage
0,1,"7,443,607",98.14%
1,2,"128,821",1.70%
2,3,"8,152",0.11%
3,4,"1,626",0.02%
4,5,576,0.01%
5,6,292,0.00%
6,7,108,0.00%
7,8,110,0.00%
8,9,66,0.00%
9,10,95,0.00%


In [19]:
generate_analytical_table_dup(var = 'index_id', case = 1)

Execution ID: fe350094-09f1-48b6-b356-fa67747674e6


,test,groups,cnt_test,cnt_index_num_voie,count_test_num_voie_pct,cnt_index_type_voie,count_test_type_voie_pct,cnt_index_commune,count_test_commune_pct,cnt_index_date,count_test_date_pct,cnt_index_admin,count_test_status_admin_pct,cnt_index_siege,count_test_siege_pct,cnt_index_cp,count_test_cp_pct
0,cas_1,False,1,"166,053",2.16%,"17,610",0.23%,"3,088",0.04%,"1,431,756",18.73%,"1,339,506",17.49%,"3,132,922",40.78%,0,0.00%
1,cas_1,Null,1,"1,025,663",13.35%,"751,806",9.91%,"110,188",1.45%,"2,372,676",31.03%,0,0.00%,0,0.00%,0,0.00%
2,cas_1,True,1,"6,443,723",83.86%,"6,676,760",88.03%,"7,330,331",96.66%,"3,757,064",49.14%,"6,246,835",81.56%,"4,500,976",58.59%,"7,443,607",98.15%
3,cas_1,False,2,"7,945",0.10%,170,0.00%,37,0.00%,"30,346",0.40%,"18,041",0.24%,"14,738",0.19%,0,0.00%
4,cas_1,Null,2,"9,410",0.12%,"9,350",0.12%,"2,347",0.03%,"37,126",0.49%,0,0.00%,0,0.00%,0,0.00%
5,cas_1,True,2,"27,237",0.35%,"118,114",1.56%,"126,437",1.67%,"9,809",0.13%,"48,208",0.63%,"27,940",0.36%,"128,821",1.70%
6,cas_1,False,3,"1,257",0.02%,3,0.00%,0,0.00%,"2,547",0.03%,"1,123",0.01%,360,0.00%,0,0.00%
7,cas_1,Null,3,739,0.01%,688,0.01%,193,0.00%,"1,383",0.02%,0,0.00%,0,0.00%,0,0.00%
8,cas_1,True,3,440,0.01%,"7,418",0.10%,"7,959",0.10%,751,0.01%,"2,975",0.04%,"3,205",0.04%,"8,152",0.11%
9,cas_1,False,4,414,0.01%,5,0.00%,0,0.00%,569,0.01%,224,0.00%,71,0.00%,0,0.00%


Analyse séquence

In [20]:
compte_dup_cas(var = 'sequence_id', case = 1)

Execution ID: a7957360-5c58-45a6-8180-246156e6b501


,count_index_id,count_duplicate_index_id,percentage
0,1,"5,954,248",87.68%
1,2,"757,519",11.16%
2,3,"50,979",0.75%
3,4,"22,484",0.33%
4,5,"1,024",0.02%
5,6,"2,324",0.03%
6,7,88,0.00%
7,8,412,0.01%
8,9,158,0.00%
9,10,155,0.00%


In [21]:
generate_analytical_table_dup(var = 'sequence_id', case = 1)

Execution ID: be94fb46-5bbc-4699-941f-efd699d75cc5


,test,groups,cnt_test,cnt_index_num_voie,count_test_num_voie_pct,cnt_index_type_voie,count_test_type_voie_pct,cnt_index_commune,count_test_commune_pct,cnt_index_date,count_test_date_pct,cnt_index_admin,count_test_status_admin_pct,cnt_index_siege,count_test_siege_pct,cnt_index_cp,count_test_cp_pct
0,cas_1,False,1,"134,015",1.95%,"15,171",0.22%,"2,585",0.04%,"1,248,131",18.00%,"1,157,852",16.78%,"2,484,293",35.57%,0,0.00%
1,cas_1,Null,1,"901,716",13.09%,"655,236",9.64%,"85,831",1.26%,"1,846,484",26.63%,0,0.00%,0,0.00%,0,0.00%
2,cas_1,True,1,"5,080,283",73.77%,"5,292,335",77.89%,"5,868,471",86.41%,"3,111,186",44.86%,"4,985,966",72.28%,"3,824,466",54.76%,"5,954,248",87.69%
3,cas_1,False,2,"19,541",0.28%,"1,205",0.02%,261,0.00%,"110,419",1.59%,"95,562",1.39%,"286,626",4.10%,0,0.00%
4,cas_1,Null,2,"64,680",0.94%,"52,506",0.77%,"13,274",0.20%,"252,554",3.64%,0,0.00%,0,0.00%,0,0.00%
5,cas_1,True,2,"623,015",9.05%,"700,482",10.31%,"743,044",10.94%,"311,612",4.49%,"595,051",8.63%,"331,402",4.74%,"757,519",11.16%
6,cas_1,False,3,"2,130",0.03%,76,0.00%,10,0.00%,"5,466",0.08%,"6,522",0.09%,"24,556",0.35%,0,0.00%
7,cas_1,Null,3,"3,505",0.05%,"2,597",0.04%,592,0.01%,"20,687",0.30%,0,0.00%,0,0.00%,0,0.00%
8,cas_1,True,3,"41,682",0.61%,"47,934",0.71%,"50,204",0.74%,"11,931",0.17%,"38,542",0.56%,"16,521",0.24%,"50,979",0.75%
9,cas_1,False,4,"1,394",0.02%,30,0.00%,5,0.00%,"3,448",0.05%,"2,303",0.03%,"7,379",0.11%,0,0.00%


## Cas de figure 2: Dissimilarité parfaite

* Definition: Aucun des mots de l’adresse de l’INPI sont égales aux mots dans l’adresse de l’INSEE
* Math definition: $\frac{|INSEE \cap INPI|}{|INSEE|+|INPI|-|INSEE \cap INPI|}$
* Query [case 2](https://eu-west-3.console.aws.amazon.com/athena/home?region=eu-west-3#query/history/4363e8b4-b3c7-4964-804f-4e66b0780a17)
* Règle: $\text{intersection} = 0 \rightarrow \text{cas 2}$

| list_inpi                               | list_insee                              | insee_except                            | intersection | union_ |
|-----------------------------------------|-----------------------------------------|-----------------------------------------|--------------|--------|
| [CHEMIN, MOUCHE]                        | [AVENUE, CHARLES, GAULLE, SAINT, GENIS] | [AVENUE, CHARLES, GAULLE, SAINT, GENIS] | 0            | 7      |
| [AVENUE, CHARLES, GAULLE, SAINT, GENIS] | [CHEMIN, MOUCHE]                        | [CHEMIN, MOUCHE]                        | 0            | 7      |

- Nombre d'observation: 974 727
    - Percentage initial: 0.08

In [22]:
cas_2 =compte_obs_cas(case= 2)

Execution ID: 1386763b-04fe-43df-bebc-3bf158564ce4


In [23]:
dic_['Cas de figure'].append(2)
dic_['Titre'].append('Exclusion parfaite')
dic_['Total'].append(cas_2)
dic_['Total cumulé'].append(cas_1 + cas_2)
dic_['pourcentage'].append(cas_2/initial_obs)
dic_['Pourcentage cumulé'].append((cas_1 +cas_2)/initial_obs)
dic_['Comment'].append("Exclusion parfaite")

In [24]:
generate_analytical_table(case = 2)

Execution ID: df47e7e7-fef1-441c-8aaf-0b314fd57ecb


,test,groups,count_test_num_voie,count_test_num_voie_pct,count_test_type_voie,count_test_type_voie_pct,count_test_commune,count_test_commune_pct,count_test_date,count_test_date_pct,count_test_status_admin,count_test_status_admin_pct,count_test_siege,count_test_siege_pct,count_test_cp,count_test_cp_pct
0,cas_2,True,"23,205",2.38%,0,0.00%,"938,971",96.36%,"256,532",26.33%,"377,981",38.79%,"560,483",57.52%,"974,444",100.00%
1,cas_2,False,"705,501",72.40%,"815,281",83.67%,"1,529",0.16%,"491,142",50.40%,"596,463",61.21%,"413,961",42.48%,0,0.00%
2,cas_2,Null,"245,738",25.22%,"159,163",16.33%,"33,944",3.48%,"226,770",23.27%,0,0.00%,0,0.00%,0,0.00%


Analyse index

In [25]:
compte_dup_cas(var = 'index_id', case = 2)

Execution ID: 28b65714-9c37-4681-9b95-b63cda502004


,count_index_id,count_duplicate_index_id,percentage
0,1,"649,128",85.71%
1,2,"74,413",9.83%
2,3,"16,586",2.19%
3,4,"5,976",0.79%
4,5,"3,002",0.40%
5,6,"1,866",0.25%
6,7,"1,310",0.17%
7,8,966,0.13%
8,9,803,0.11%
9,10,554,0.07%


In [26]:
generate_analytical_table_dup(var = 'index_id', case = 2)

Execution ID: 7e44b072-fda9-4305-8b7d-2fe0ca2723c5


,test,groups,cnt_test,cnt_index_num_voie,count_test_num_voie_pct,cnt_index_type_voie,count_test_type_voie_pct,cnt_index_commune,count_test_commune_pct,cnt_index_date,count_test_date_pct,cnt_index_admin,count_test_status_admin_pct,cnt_index_siege,count_test_siege_pct,cnt_index_cp,count_test_cp_pct
0,cas_2,False,1,"471,966",60.04%,"539,293",69.70%,"1,173",0.16%,"303,762",38.58%,"447,680",55.64%,"356,073",45.33%,0,0.00%
1,cas_2,Null,1,"197,741",25.15%,"135,749",17.54%,"21,165",2.80%,"175,269",22.26%,0,0.00%,0,0.00%,0,0.00%
2,cas_2,True,1,"22,342",2.84%,0,0.00%,"626,790",82.83%,"220,078",27.95%,"266,958",33.18%,"342,898",43.66%,"649,128",85.78%
3,cas_2,False,2,"52,709",6.70%,"61,782",7.98%,95,0.01%,"38,873",4.94%,"42,923",5.33%,"20,535",2.61%,0,0.00%
4,cas_2,Null,2,"12,860",1.64%,"6,556",0.85%,"2,359",0.31%,"15,049",1.91%,0,0.00%,0,0.00%,0,0.00%
5,cas_2,True,2,330,0.04%,0,0.00%,"71,959",9.51%,"4,860",0.62%,"19,423",2.41%,"37,218",4.74%,"74,413",9.83%
6,cas_2,False,3,"11,500",1.46%,"13,869",1.79%,16,0.00%,"10,025",1.27%,"7,760",0.96%,"3,266",0.42%,0,0.00%
7,cas_2,Null,3,"2,274",0.29%,"1,019",0.13%,646,0.09%,"2,678",0.34%,0,0.00%,0,0.00%,0,0.00%
8,cas_2,True,3,44,0.01%,0,0.00%,"15,924",2.10%,"1,561",0.20%,"5,668",0.70%,"10,196",1.30%,"16,586",2.19%
9,cas_2,False,4,"4,312",0.55%,"5,098",0.66%,6,0.00%,"3,757",0.48%,"2,722",0.34%,712,0.09%,0,0.00%


Analyse séquence

In [27]:
compte_dup_cas(var = 'sequence_id', case = 2)

Execution ID: 49b7f8fb-5e1a-4895-82cb-dcc237bb3966


,count_index_id,count_duplicate_index_id,percentage
0,1,"520,540",77.04%
1,2,"111,584",16.51%
2,3,"18,157",2.69%
3,4,"11,952",1.77%
4,5,"2,531",0.37%
5,6,"3,686",0.55%
6,7,"1,009",0.15%
7,8,"1,470",0.22%
8,9,732,0.11%
9,10,751,0.11%


In [28]:
generate_analytical_table_dup(var = 'sequence_id', case = 2)

Execution ID: 4880825b-df8c-436d-98b1-b1de0f2622f4


,test,groups,cnt_test,cnt_index_num_voie,count_test_num_voie_pct,cnt_index_type_voie,count_test_type_voie_pct,cnt_index_commune,count_test_commune_pct,cnt_index_date,count_test_date_pct,cnt_index_admin,count_test_status_admin_pct,cnt_index_siege,count_test_siege_pct,cnt_index_cp,count_test_cp_pct
0,cas_2,False,1,"368,582",52.50%,"427,006",61.78%,966,0.14%,"235,961",32.86%,"370,758",51.09%,"303,738",42.27%,0,0.00%
1,cas_2,Null,1,"168,377",23.98%,"115,860",16.76%,"16,953",2.51%,"158,496",22.07%,0,0.00%,0,0.00%,0,0.00%
2,cas_2,True,1,"18,776",2.67%,0,0.00%,"503,381",74.54%,"190,345",26.51%,"213,954",29.48%,"286,647",39.89%,"520,540",77.14%
3,cas_2,False,2,"83,689",11.92%,"94,503",13.67%,173,0.03%,"59,916",8.34%,"68,413",9.43%,"40,608",5.65%,0,0.00%
4,cas_2,Null,2,"23,152",3.30%,"14,126",2.04%,"3,717",0.55%,"20,212",2.81%,0,0.00%,0,0.00%,0,0.00%
5,cas_2,True,2,"1,835",0.26%,0,0.00%,"107,549",15.93%,"17,567",2.45%,"36,886",5.08%,"53,049",7.38%,"111,584",16.54%
6,cas_2,False,3,"13,224",1.88%,"15,416",2.23%,20,0.00%,"9,488",1.32%,"9,522",1.31%,"5,235",0.73%,0,0.00%
7,cas_2,Null,3,"2,789",0.40%,"1,444",0.21%,577,0.09%,"3,486",0.49%,0,0.00%,0,0.00%,0,0.00%
8,cas_2,True,3,156,0.02%,0,0.00%,"17,519",2.59%,"1,849",0.26%,"5,885",0.81%,"9,617",1.34%,"18,157",2.69%
9,cas_2,False,4,"8,758",1.25%,"10,163",1.47%,9,0.00%,"7,191",1.00%,"5,219",0.72%,"1,665",0.23%,0,0.00%


## Cas de figure 3: Intersection parfaite INPI

* Definition:  Tous les mots dans l’adresse de l’INPI  sont contenus dans l’adresse de l’INSEE
* Math définition: $\frac{|INPI|}{|INSEE \cap INPI|}  \text{  = 1 and }|INSEE \cap INPI| <> |INSEE \cup INPI|$
* Query [case 3](https://eu-west-3.console.aws.amazon.com/athena/home?region=eu-west-3#query/history/7fb420a1-5f50-4256-a2ba-b8c7c2b63c9b)
* Règle: $|\text{list_inpi}|= \text{intersection}  \text{  = 1 and }\text{intersection} \neq  \text{union} \rightarrow \text{cas 3}$

| list_inpi                    | list_insee                                               | insee_except                            | intersection | union_ |
|------------------------------|----------------------------------------------------------|-----------------------------------------|--------------|--------|
| [ALLEE, BERLIOZ]             | [ALLEE, BERLIOZ, CHEZ, MME, IDALI]                       | [CHEZ, MME, IDALI]                      | 2            | 5      |
| [RUE, MAI, OLONNE, SUR, MER] | [RUE, HUIT, MAI, OLONNE, SUR, MER]                       | [HUIT]                                  | 5            | 6      |
| [RUE, CAMILLE, CLAUDEL]      | [RUE, CAMILLE, CLAUDEL, VITRE]                           | [VITRE]                                 | 3            | 4      |
| [ROUTE, D, ESLETTES]         | [ROUTE, D, ESLETTES, A]                                  | [A]                                     | 3            | 4      |
| [AVENUE, MAI]                | [AVENUE, HUIT, MAI]                                      | [HUIT]                                  | 2            | 3      |
| [RUE, SOUS, DINE]            | [RUE, SOUS, DINE, RES, SOCIALE, HENRIETTE, D, ANGEVILLE] | [RES, SOCIALE, HENRIETTE, D, ANGEVILLE] | 3            | 8      |

- Nombre d'observation: 407404
    - Percentage initial: 0.03

In [29]:
cas_3 = compte_obs_cas(case= 3)

Execution ID: bad295e8-0f77-4b2f-8884-474c44b0b971


In [30]:
dic_['Cas de figure'].append(3)
dic_['Titre'].append('Match partiel parfait')
dic_['Total'].append(cas_3)
dic_['Total cumulé'].append(cas_1 + cas_2 +cas_3)
dic_['pourcentage'].append(cas_3/initial_obs)
dic_['Pourcentage cumulé'].append((cas_1 + cas_2 +cas_3)/initial_obs)
dic_['Comment'].append("Match partiel parfait")

In [31]:
generate_analytical_table(case = 3)

Execution ID: 85a93ebb-4420-4fc2-8330-1d49d884a7a3


,test,groups,count_test_num_voie,count_test_num_voie_pct,count_test_type_voie,count_test_type_voie_pct,count_test_commune,count_test_commune_pct,count_test_date,count_test_date_pct,count_test_status_admin,count_test_status_admin_pct,count_test_siege,count_test_siege_pct,count_test_cp,count_test_cp_pct
0,cas_3,True,"259,463",63.69%,"289,312",71.01%,"393,070",96.48%,"181,981",44.67%,"320,406",78.65%,"244,121",59.92%,"407,404",100.00%
1,cas_3,Null,"110,931",27.23%,"37,878",9.30%,"14,029",3.44%,"119,869",29.42%,0,0.00%,0,0.00%,0,0.00%
2,cas_3,False,"37,010",9.08%,"80,214",19.69%,305,0.07%,"105,554",25.91%,"86,998",21.35%,"163,283",40.08%,0,0.00%


Analyse index

In [32]:
compte_dup_cas(var = 'index_id', case = 3)

Execution ID: ac0c97f7-f1d6-4d2b-86ab-5093ec7aaa53


,count_index_id,count_duplicate_index_id,percentage
0,1,"387,589",97.94%
1,2,"6,915",1.75%
2,3,721,0.18%
3,4,246,0.06%
4,5,72,0.02%
5,6,69,0.02%
6,7,9,0.00%
7,8,28,0.01%
8,9,13,0.00%
9,10,13,0.00%


In [33]:
generate_analytical_table_dup(var = 'index_id', case = 3)

Execution ID: e3db1c39-a3c2-465c-bd4c-03f17ae7e290


,test,groups,cnt_test,cnt_index_num_voie,count_test_num_voie_pct,cnt_index_type_voie,count_test_type_voie_pct,cnt_index_commune,count_test_commune_pct,cnt_index_date,count_test_date_pct,cnt_index_admin,count_test_status_admin_pct,cnt_index_siege,count_test_siege_pct,cnt_index_cp,count_test_cp_pct
0,cas_3,False,1,"34,098",8.54%,"78,038",19.69%,298,0.08%,"97,447",24.44%,"83,528",20.92%,"161,087",40.27%,0,0.00%
1,cas_3,Null,1,"105,140",26.33%,"34,949",8.82%,"13,468",3.40%,"115,922",29.07%,0,0.00%,0,0.00%,0,0.00%
2,cas_3,True,1,"254,858",63.83%,"275,559",69.54%,"373,823",94.47%,"179,688",45.07%,"310,540",77.77%,"234,578",58.65%,"387,589",97.95%
3,cas_3,False,2,"1,018",0.25%,884,0.22%,2,0.00%,"2,448",0.61%,"1,197",0.30%,803,0.20%,0,0.00%
4,cas_3,Null,2,"1,563",0.39%,659,0.17%,222,0.06%,"1,571",0.39%,0,0.00%,0,0.00%,0,0.00%
5,cas_3,True,2,"1,704",0.43%,"5,038",1.27%,"6,691",1.69%,686,0.17%,"3,120",0.78%,"2,621",0.66%,"6,915",1.75%
6,cas_3,False,3,140,0.04%,100,0.03%,1,0.00%,324,0.08%,134,0.03%,64,0.02%,0,0.00%
7,cas_3,Null,3,204,0.05%,94,0.02%,17,0.00%,122,0.03%,0,0.00%,0,0.00%,0,0.00%
8,cas_3,True,3,134,0.03%,476,0.12%,703,0.18%,74,0.02%,424,0.11%,402,0.10%,721,0.18%
9,cas_3,False,4,42,0.01%,18,0.00%,0,0.00%,160,0.04%,41,0.01%,34,0.01%,0,0.00%


Analyse séquence

In [34]:
compte_dup_cas(var = 'sequence_id', case = 3)

Execution ID: 18dad1f7-43a9-4675-a86f-d45f491e6bc7


,count_index_id,count_duplicate_index_id,percentage
0,1,"325,612",89.75%
1,2,"33,627",9.27%
2,3,"2,039",0.56%
3,4,"1,061",0.29%
4,5,76,0.02%
5,6,172,0.05%
6,7,11,0.00%
7,8,48,0.01%
8,9,11,0.00%
9,10,18,0.00%


In [35]:
generate_analytical_table_dup(var = 'sequence_id', case = 3)

Execution ID: 3799f34d-ccfc-419a-aa2c-3735d1a4b4c5


,test,groups,cnt_test,cnt_index_num_voie,count_test_num_voie_pct,cnt_index_type_voie,count_test_type_voie_pct,cnt_index_commune,count_test_commune_pct,cnt_index_date,count_test_date_pct,cnt_index_admin,count_test_status_admin_pct,cnt_index_siege,count_test_siege_pct,cnt_index_cp,count_test_cp_pct
0,cas_3,False,1,"28,900",7.89%,"68,144",18.75%,254,0.07%,"84,847",23.03%,"73,018",19.95%,"136,908",37.05%,0,0.00%
1,cas_3,Null,1,"92,497",25.26%,"30,028",8.26%,"10,777",2.97%,"100,931",27.40%,0,0.00%,0,0.00%,0,0.00%
2,cas_3,True,1,"209,973",57.34%,"228,459",62.87%,"314,788",86.75%,"149,273",40.52%,"257,872",70.46%,"200,982",54.38%,"325,612",89.76%
3,cas_3,False,2,"3,072",0.84%,"5,281",1.45%,21,0.01%,"7,769",2.11%,"5,706",1.56%,"11,359",3.07%,0,0.00%
4,cas_3,Null,2,"6,963",1.90%,"2,770",0.76%,"1,440",0.40%,"8,178",2.22%,0,0.00%,0,0.00%,0,0.00%
5,cas_3,True,2,"21,798",5.95%,"25,272",6.95%,"32,097",8.85%,"14,779",4.01%,"26,323",7.19%,"17,581",4.76%,"33,627",9.27%
6,cas_3,False,3,279,0.08%,308,0.08%,3,0.00%,505,0.14%,397,0.11%,767,0.21%,0,0.00%
7,cas_3,Null,3,499,0.14%,191,0.05%,62,0.02%,387,0.11%,0,0.00%,0,0.00%,0,0.00%
8,cas_3,True,3,"1,100",0.30%,"1,497",0.41%,"1,964",0.54%,622,0.17%,"1,560",0.43%,896,0.24%,"2,039",0.56%
9,cas_3,False,4,164,0.04%,96,0.03%,0,0.00%,382,0.10%,157,0.04%,228,0.06%,0,0.00%


## Cas de figure 4: Intersection parfaite INSEE

* Definition:  Tous les mots dans l’adresse de l’INSEE  sont contenus dans l’adresse de l’INPI
* Math definition: $\frac{|INSEE|}{|INSEE \cap INPI|}  \text{  = 1 and }|INSEE \cap INPI| <> |INSEE \cup INPI|$
* Query [case 4](https://eu-west-3.console.aws.amazon.com/athena/home?region=eu-west-3#query/history/65344bf4-8999-4ddb-a65e-11bb825f5f40)
* Règle: $|\text{list_insee}|= \text{intersection}  \text{  = 1 and }\text{intersection} \neq  \text{union} \rightarrow \text{cas 4}$

| list_inpi                                                 | list_insee                                      | insee_except | intersection | union_ |
|-----------------------------------------------------------|-------------------------------------------------|--------------|--------------|--------|
| [ROUTE, D, ENGHIEN]                                       | [ROUTE, ENGHIEN]                                | []           | 2            | 3      |
| [ZAC, PARC, D, ACTIVITE, PARIS, EST, ALLEE, LECH, WALESA] | [ALLEE, LECH, WALESA, ZAC, PARC, ACTIVITE, EST] | []           | 7            | 9      |
| [LIEU, DIT, PADER, QUARTIER, RIBERE]                      | [LIEU, DIT, RIBERE]                             | []           | 3            | 5      |
| [A, BOULEVARD, CONSTANTIN, DESCAT]                        | [BOULEVARD, CONSTANTIN, DESCAT]                 | []           | 3            | 4      |
| [RUE, MENILMONTANT, BP]                                   | [RUE, MENILMONTANT]                             | []           | 2            | 3      |

- Nombre d'observation: 558992
    - Percentage initial: 0.05

In [36]:
cas_4 = compte_obs_cas(case= 4)

Execution ID: c584a2ca-57de-4972-9f82-4ca5726cfcd4


In [37]:
dic_['Cas de figure'].append(4)
dic_['Titre'].append('Match partiel parfait')
dic_['Total'].append(cas_4)
dic_['Total cumulé'].append(cas_1 + cas_2 + cas_3 + cas_4)
dic_['pourcentage'].append(cas_4/initial_obs)
dic_['Pourcentage cumulé'].append((cas_1 + cas_2 + cas_3 + cas_4) / initial_obs)
dic_['Comment'].append("Match partiel parfait")

In [38]:
generate_analytical_table(case = 4)

Execution ID: 36a3b489-2c50-4395-ab86-1148d86b647f


,test,groups,count_test_num_voie,count_test_num_voie_pct,count_test_type_voie,count_test_type_voie_pct,count_test_commune,count_test_commune_pct,count_test_date,count_test_date_pct,count_test_status_admin,count_test_status_admin_pct,count_test_siege,count_test_siege_pct,count_test_cp,count_test_cp_pct
0,cas_4,False,"36,724",6.57%,"11,503",2.06%,"2,751",0.49%,"125,430",22.44%,"123,869",22.16%,"210,573",37.67%,0,0.00%
1,cas_4,Null,"198,374",35.49%,"168,672",30.17%,"22,870",4.09%,"170,830",30.56%,0,0.00%,0,0.00%,0,0.00%
2,cas_4,True,"323,894",57.94%,"378,817",67.77%,"533,371",95.42%,"262,732",47.00%,"435,123",77.84%,"348,419",62.33%,"558,992",100.00%


Analyse index

In [39]:
compte_dup_cas(var = 'index_id', case = 4)

Execution ID: 6fbda57b-3381-4feb-b9cb-1e8692cf23b0


,count_index_id,count_duplicate_index_id,percentage
0,1,"525,832",97.75%
1,2,"10,572",1.97%
2,3,830,0.15%
3,4,236,0.04%
4,5,61,0.01%
5,6,43,0.01%
6,7,39,0.01%
7,8,28,0.01%
8,9,15,0.00%
9,10,55,0.01%


In [40]:
generate_analytical_table_dup(var = 'index_id', case = 4)

Execution ID: 4ca2adf9-25fb-4d67-87c9-4bbfce97ea43


,test,groups,cnt_test,cnt_index_num_voie,count_test_num_voie_pct,cnt_index_type_voie,count_test_type_voie_pct,cnt_index_commune,count_test_commune_pct,cnt_index_date,count_test_date_pct,cnt_index_admin,count_test_status_admin_pct,cnt_index_siege,count_test_siege_pct,cnt_index_cp,count_test_cp_pct
0,cas_4,False,1,"32,392",5.95%,"10,960",2.03%,"2,652",0.49%,"116,001",21.36%,"119,180",21.91%,"209,260",38.34%,0,0.00%
1,cas_4,Null,1,"185,043",34.02%,"155,929",28.93%,"21,876",4.07%,"166,561",30.67%,0,0.00%,0,0.00%,0,0.00%
2,cas_4,True,1,"320,206",58.86%,"361,456",67.05%,"501,304",93.22%,"253,205",46.62%,"418,014",76.85%,"331,856",60.81%,"525,832",97.78%
3,cas_4,False,2,"1,416",0.26%,206,0.04%,43,0.01%,"3,278",0.60%,"1,733",0.32%,514,0.09%,0,0.00%
4,cas_4,Null,2,"2,421",0.45%,"2,159",0.40%,374,0.07%,"1,862",0.34%,0,0.00%,0,0.00%,0,0.00%
5,cas_4,True,2,"1,651",0.30%,"7,062",1.31%,"10,155",1.89%,"1,271",0.23%,"4,053",0.75%,"3,154",0.58%,"10,572",1.97%
6,cas_4,False,3,244,0.04%,18,0.00%,3,0.00%,342,0.06%,124,0.02%,48,0.01%,0,0.00%
7,cas_4,Null,3,164,0.03%,174,0.03%,32,0.01%,92,0.02%,0,0.00%,0,0.00%,0,0.00%
8,cas_4,True,3,52,0.01%,607,0.11%,795,0.15%,90,0.02%,409,0.08%,491,0.09%,830,0.15%
9,cas_4,False,4,58,0.01%,9,0.00%,1,0.00%,112,0.02%,42,0.01%,8,0.00%,0,0.00%


Analyse séquence

In [41]:
compte_dup_cas(var = 'sequence_id', case = 4)

Execution ID: 329766a9-ee1e-43a6-aff0-b43037f4bfb9


,count_index_id,count_duplicate_index_id,percentage
0,1,"436,201",88.87%
1,2,"50,531",10.29%
2,3,"2,232",0.45%
3,4,"1,237",0.25%
4,5,63,0.01%
5,6,214,0.04%
6,7,31,0.01%
7,8,59,0.01%
8,9,19,0.00%
9,10,56,0.01%


In [42]:
generate_analytical_table_dup(var = 'sequence_id', case = 4)

Execution ID: 7919c8ab-a73a-471d-adde-7967f9c28d88


,test,groups,cnt_test,cnt_index_num_voie,count_test_num_voie_pct,cnt_index_type_voie,count_test_type_voie_pct,cnt_index_commune,count_test_commune_pct,cnt_index_date,count_test_date_pct,cnt_index_admin,count_test_status_admin_pct,cnt_index_siege,count_test_siege_pct,cnt_index_cp,count_test_cp_pct
0,cas_4,False,1,"27,443",5.52%,"9,552",1.94%,"2,333",0.48%,"98,659",19.83%,"102,335",20.58%,"176,843",35.20%,0,0.00%
1,cas_4,Null,1,"158,717",31.91%,"133,549",27.13%,"17,507",3.57%,"141,741",28.49%,0,0.00%,0,0.00%,0,0.00%
2,cas_4,True,1,"261,823",52.63%,"296,087",60.14%,"417,002",84.93%,"207,098",41.62%,"344,936",69.38%,"280,594",55.84%,"436,201",88.90%
3,cas_4,False,2,"3,259",0.66%,819,0.17%,187,0.04%,"10,804",2.17%,"9,342",1.88%,"15,441",3.07%,0,0.00%
4,cas_4,Null,2,"14,357",2.89%,"12,318",2.50%,"2,384",0.49%,"13,535",2.72%,0,0.00%,0,0.00%,0,0.00%
5,cas_4,True,2,"28,760",5.78%,"36,248",7.36%,"47,689",9.71%,"22,973",4.62%,"37,427",7.53%,"26,846",5.34%,"50,531",10.30%
6,cas_4,False,3,322,0.06%,37,0.01%,12,0.00%,540,0.11%,428,0.09%,719,0.14%,0,0.00%
7,cas_4,Null,3,591,0.12%,512,0.10%,64,0.01%,371,0.07%,0,0.00%,0,0.00%,0,0.00%
8,cas_4,True,3,"1,007",0.20%,"1,596",0.32%,"2,137",0.44%,685,0.14%,"1,520",0.31%,"1,034",0.21%,"2,232",0.45%
9,cas_4,False,4,230,0.05%,32,0.01%,2,0.00%,410,0.08%,189,0.04%,119,0.02%,0,0.00%


## Cas de figure 5: Cardinality exception parfaite INSEE INPI, intersection positive

* Definition:  L’adresse de l’INPI contient des mots de l’adresse de l’INPI et la cardinality des mots non présents dans les deux adresses est équivalente
* Math definition: $|INPI|-|INPI \cap INSEE| = |INSEE|-|INPI \cap INSEE|$
* Query [case 5](https://eu-west-3.console.aws.amazon.com/athena/home?region=eu-west-3#query/history/fec67222-3a7b-4bfb-af20-dd70d82932e3)
* Règle: $|\text{insee_except}| = |\text{inpi_except}| \text{ and } \text{intersection} > 0 \rightarrow \text{cas 5}$

| list_inpi                                                                                  | list_insee                                                                              | insee_except | inpi_except  | intersection | union_ |
|--------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------|--------------|--------------|--------------|--------|
| [AVENUE, GEORGES, VACHER, C, A, SAINTE, VICTOIRE, IMMEUBLE, CCE, CD, ZI, ROUSSET, PEYNIER] | [AVENUE, GEORGES, VACHER, C, A, STE, VICTOIRE, IMMEUBLE, CCE, CD, ZI, ROUSSET, PEYNIER] | [STE]        | [SAINTE]     | 12           | 14     |
| [BIS, AVENUE, PAUL, DOUMER, RES, SAINT, MARTIN, BAT, D, C, O, M, ROSSI]                    | [BIS, AVENUE, PAUL, DOUMER, RES, ST, MARTIN, BAT, D, C, O, M, ROSSI]                    | [ST]         | [SAINT]      | 12           | 14     |
| [ROUTE, DEPARTEMENTALE, CHEZ, SOREME, CENTRE, COMMERCIAL, L, OCCITAN, PLAN, OCCIDENTAL]    | [ROUTE, DEPARTEMENTALE, CHEZ, SOREME, CENTRE, COMMERCIAL, L, OCCITAN, PLAN, OC]         | [OC]         | [OCCIDENTAL] | 9            | 11     |
| [LIEU, DIT, FOND, CHAMP, MALTON, PARC, EOLIEN, SUD, MARNE, PDL]                            | [LIEU, DIT, FONDD, CHAMP, MALTON, PARC, EOLIEN, SUD, MARNE, PDL]                        | [FONDD]      | [FOND]       | 9            | 11     |
| [AVENUE, ROBERT, BRUN, ZI, CAMP, LAURENT, LOT, NUMERO, ST, BERNARD]                        | [AVENUE, ROBERT, BRUN, ZI, CAMP, LAURENT, LOT, ST, BERNARD, N]                          | [N]          | [NUMERO]     | 9            | 11     |
| [PLACE, MARCEL, DASSAULT, PARC, D, ACTIVITES, TY, NEHUE, BATIMENT, H]                      | [PLACE, MARCEL, DASSAULT, PARC, D, ACTIVITES, TY, NEHUE, BAT, H]                        | [BAT]        | [BATIMENT]   | 9            | 11     |

- Nombre d'observation: 1056406
    - Percentage initial: 0.09

In [43]:
cas_5 = compte_obs_cas(case= 5)

Execution ID: 2581d3a5-8cd5-4e23-96b1-ee9d2d629a2e


In [44]:
dic_['Cas de figure'].append(5)
dic_['Titre'].append('Match partiel compliqué')
dic_['Total'].append(cas_5)
dic_['Total cumulé'].append(cas_1 + cas_2 + cas_3 + cas_4 + cas_5)
dic_['pourcentage'].append(cas_5/initial_obs)
dic_['Pourcentage cumulé'].append((cas_1 + cas_2 + cas_3 + cas_4 + cas_5)/initial_obs)
dic_['Comment'].append("Match partiel compliqué")

In [45]:
generate_analytical_table(case = 5)

Execution ID: e0075999-8989-4390-9a8c-1c22d1292ad1


,test,groups,count_test_num_voie,count_test_num_voie_pct,count_test_type_voie,count_test_type_voie_pct,count_test_commune,count_test_commune_pct,count_test_date,count_test_date_pct,count_test_status_admin,count_test_status_admin_pct,count_test_siege,count_test_siege_pct,count_test_cp,count_test_cp_pct
0,cas_5,True,"631,061",59.74%,"712,321",67.43%,"1,010,098",95.62%,"463,997",43.92%,"727,118",68.83%,"628,399",59.48%,"1,056,406",100.00%
1,cas_5,Null,"188,775",17.87%,"109,594",10.37%,"45,352",4.29%,"296,638",28.08%,0,0.00%,0,0.00%,0,0.00%
2,cas_5,False,"236,570",22.39%,"234,491",22.20%,956,0.09%,"295,771",28.00%,"329,288",31.17%,"428,007",40.52%,0,0.00%


Analyse index

In [46]:
compte_dup_cas(var = 'index_id', case = 5)

Execution ID: d9d54c11-2b4e-4169-b934-0c95dfb525d9


,count_index_id,count_duplicate_index_id,percentage
0,1,"943,789",95.76%
1,2,"33,429",3.39%
2,3,"4,467",0.45%
3,4,"1,474",0.15%
4,5,741,0.08%
5,6,470,0.05%
6,7,203,0.02%
7,8,134,0.01%
8,9,115,0.01%
9,10,124,0.01%


In [47]:
generate_analytical_table_dup(var = 'index_id', case = 5)

Execution ID: 394da01b-f39d-400f-862f-191ad92a6446


,test,groups,cnt_test,cnt_index_num_voie,count_test_num_voie_pct,cnt_index_type_voie,count_test_type_voie_pct,cnt_index_commune,count_test_commune_pct,cnt_index_date,count_test_date_pct,cnt_index_admin,count_test_status_admin_pct,cnt_index_siege,count_test_siege_pct,cnt_index_cp,count_test_cp_pct
0,cas_5,False,1,"188,602",18.81%,"226,028",22.84%,921,0.09%,"251,449",25.15%,"299,116",29.75%,"416,802",41.54%,0,0.00%
1,cas_5,Null,1,"163,656",16.32%,"96,870",9.79%,"42,385",4.30%,"277,862",27.79%,0,0.00%,0,0.00%,0,0.00%
2,cas_5,True,1,"624,127",62.24%,"628,397",63.50%,"900,483",91.40%,"440,840",44.09%,"679,414",67.56%,"561,655",55.97%,"943,789",95.79%
3,cas_5,False,2,"13,488",1.35%,"3,430",0.35%,16,0.00%,"12,528",1.25%,"10,419",1.04%,"4,527",0.45%,0,0.00%
4,cas_5,Null,2,"3,423",0.34%,"2,111",0.21%,"1,052",0.11%,"7,396",0.74%,0,0.00%,0,0.00%,0,0.00%
5,cas_5,True,2,"3,050",0.30%,"25,340",2.56%,"32,361",3.28%,"3,133",0.31%,"10,635",1.06%,"13,699",1.37%,"33,429",3.39%
6,cas_5,False,3,"2,521",0.25%,334,0.03%,1,0.00%,"2,249",0.22%,"1,383",0.14%,442,0.04%,0,0.00%
7,cas_5,Null,3,654,0.07%,411,0.04%,134,0.01%,626,0.06%,0,0.00%,0,0.00%,0,0.00%
8,cas_5,True,3,166,0.02%,"3,422",0.35%,"4,332",0.44%,641,0.06%,"1,950",0.19%,"3,026",0.30%,"4,467",0.45%
9,cas_5,False,4,932,0.09%,73,0.01%,0,0.00%,818,0.08%,393,0.04%,66,0.01%,0,0.00%


Analyse séquence

In [48]:
compte_dup_cas(var = 'sequence_id', case = 5)

Execution ID: 71505690-94c8-4cc4-b4aa-2e916fa949af


,count_index_id,count_duplicate_index_id,percentage
0,1,"778,442",87.03%
1,2,"99,188",11.09%
2,3,"8,242",0.92%
3,4,"5,222",0.58%
4,5,692,0.08%
5,6,"1,092",0.12%
6,7,177,0.02%
7,8,336,0.04%
8,9,137,0.02%
9,10,175,0.02%


In [49]:
generate_analytical_table_dup(var = 'sequence_id', case = 5)

Execution ID: 95bdf595-cba5-47c9-b674-24e0b5212193


,test,groups,cnt_test,cnt_index_num_voie,count_test_num_voie_pct,cnt_index_type_voie,count_test_type_voie_pct,cnt_index_commune,count_test_commune_pct,cnt_index_date,count_test_date_pct,cnt_index_admin,count_test_status_admin_pct,cnt_index_siege,count_test_siege_pct,cnt_index_cp,count_test_cp_pct
0,cas_5,False,1,"145,039",15.92%,"200,149",22.24%,786,0.09%,"212,025",23.07%,"254,984",27.85%,"348,015",37.73%,0,0.00%
1,cas_5,Null,1,"143,012",15.70%,"82,176",9.13%,"36,670",4.10%,"235,966",25.67%,0,0.00%,0,0.00%,0,0.00%
2,cas_5,True,1,"518,142",56.88%,"505,520",56.18%,"741,522",82.91%,"370,956",40.36%,"556,734",60.80%,"480,134",52.06%,"778,442",87.07%
3,cas_5,False,2,"28,470",3.13%,"14,735",1.64%,75,0.01%,"28,253",3.07%,"27,956",3.05%,"33,730",3.66%,0,0.00%
4,cas_5,Null,2,"12,116",1.33%,"8,299",0.92%,"3,488",0.39%,"24,442",2.66%,0,0.00%,0,0.00%,0,0.00%
5,cas_5,True,2,"50,331",5.53%,"73,282",8.14%,"95,441",10.67%,"35,470",3.86%,"62,808",6.86%,"47,481",5.15%,"99,188",11.09%
6,cas_5,False,3,"3,419",0.38%,819,0.09%,4,0.00%,"2,584",0.28%,"2,648",0.29%,"2,838",0.31%,0,0.00%
7,cas_5,Null,3,"1,024",0.11%,688,0.08%,214,0.02%,"2,028",0.22%,0,0.00%,0,0.00%,0,0.00%
8,cas_5,True,3,"2,857",0.31%,"6,412",0.71%,"8,005",0.90%,"1,701",0.19%,"4,644",0.51%,"4,015",0.44%,"8,242",0.92%
9,cas_5,False,4,"2,582",0.28%,403,0.04%,2,0.00%,"1,870",0.20%,"1,201",0.13%,708,0.08%,0,0.00%


## Cas de figure 6: Cardinality exception INSEE supérieure INPI, intersection positive 

* Definition:  L’adresse de l’INPI contient des mots de l’adresse de l’INPI et la cardinality des mots non présents dans l’adresse de l’INSEE est supérieure à la cardinality de l’adresse de l’INPI
* Math definition: $|INPI|-|INPI \cap INSEE| < |INSEE|-|INPI \cap INSEE|$
* Query [case 6](https://eu-west-3.console.aws.amazon.com/athena/home?region=eu-west-3#query/history/9bdce567-5871-4a5a-add4-d5cca6a83528)
* Règle: $|\text{insee_except}| > |\text{inpi_except}| \text{ and } \text{intersection} > 0 \rightarrow \text{cas 6}$

| list_inpi                                                                         | list_insee                                                                               | insee_except          | inpi_except   | intersection | union_ |
|-----------------------------------------------------------------------------------|------------------------------------------------------------------------------------------|-----------------------|---------------|--------------|--------|
| [AVENUE, AUGUSTE, PICARD, POP, UP, TOURVILLE, CC, EMPLACEMENT, DIT, PRECAIRE, N]  | [AVENUE, AUGUSTE, PICARD, POP, UP, TOURVILL, CC, TOURVILLE, EMPLACEMT, DIT, PRECAIRE, N] | [TOURVILL, EMPLACEMT] | [EMPLACEMENT] | 10           | 13     |
| [ROUTE, COTE, D, AZUR, C, O, TENERGIE, ARTEPARC, MEYREUIL, BAT, A]                | [ROUTE, C, O, TENERGIE, ARTEPARC, MEYREUI, BAT, A, RTE, COTE, D, AZUR]                   | [MEYREUI, RTE]        | [MEYREUIL]    | 10           | 13     |
| [C, O, TENERGIE, ARTEPARC, MEYREUIL, BATIMENT, A, ROUTE, COTE, D, AZUR]           | [ROUTE, C, O, TENERGIE, ARTEPARC, MEYREUI, BATIMENT, A, RTE, COTE, D, AZUR]              | [MEYREUI, RTE]        | [MEYREUIL]    | 10           | 13     |
| [LOTISSEMENT, VANGA, DI, L, ORU, VILLA, FRANCK, TINA, CHEZ, COLOMBANI, CHRISTIAN] | [LIEU, DIT, VANGA, DI, L, ORU, VILLA, FRANCK, TINA, CHEZ, COLOMBANI, CHRISTIAN]          | [LIEU, DIT]           | [LOTISSEMENT] | 10           | 13     |
| [AVENUE, DECLARATION, DROITS, HOMME, RES, CLOS, ST, MAMET, BAT, C, APPT]          | [AVENUE, DECL, DROITS, L, HOMME, RES, CLOS, ST, MAMET, BAT, C, APPT]                     | [DECL, L]             | [DECLARATION] | 10           | 13     |

- Nombre d'observation: 361242
    - Percentage initial: 0.03

In [50]:
cas_6 = compte_obs_cas(case= 6)

Execution ID: b400044a-d860-4b2a-a7c2-18e65dcce100


In [51]:
dic_['Cas de figure'].append(6)
dic_['Titre'].append('Match partiel compliqué')
dic_['Total'].append(cas_6)
dic_['Total cumulé'].append(cas_1 + cas_2 +cas_3 +cas_4 + cas_5 +cas_6)
dic_['pourcentage'].append(cas_6/initial_obs)
dic_['Pourcentage cumulé'].append((cas_1 + cas_2 +cas_3 +cas_4 + cas_5 +cas_6)/initial_obs)
dic_['Comment'].append("Match partiel compliqué")

In [52]:
generate_analytical_table(case = 6)

Execution ID: 81aaa3ad-b713-4475-bcbd-9c500d9eccd7


,test,groups,count_test_num_voie,count_test_num_voie_pct,count_test_type_voie,count_test_type_voie_pct,count_test_commune,count_test_commune_pct,count_test_date,count_test_date_pct,count_test_status_admin,count_test_status_admin_pct,count_test_siege,count_test_siege_pct,count_test_cp,count_test_cp_pct
0,cas_6,True,"118,441",25.38%,"287,344",61.57%,"444,419",95.23%,"163,465",35.03%,"260,679",55.86%,"289,877",62.12%,"466,671",100.00%
1,cas_6,False,"199,235",42.69%,"92,162",19.75%,"1,177",0.25%,"179,577",38.48%,"205,992",44.14%,"176,794",37.88%,0,0.00%
2,cas_6,Null,"148,995",31.93%,"87,165",18.68%,"21,075",4.52%,"123,629",26.49%,0,0.00%,0,0.00%,0,0.00%


Analyse index

In [53]:
compte_dup_cas(var = 'index_id', case = 6)

Execution ID: 8750ec6b-c8f1-416c-97c5-a1bf554d797f


,count_index_id,count_duplicate_index_id,percentage
0,1,"367,508",91.51%
1,2,"25,116",6.25%
2,3,"4,395",1.09%
3,4,"1,632",0.41%
4,5,824,0.21%
5,6,498,0.12%
6,7,377,0.09%
7,8,217,0.05%
8,9,155,0.04%
9,10,130,0.03%


In [54]:
generate_analytical_table_dup(var = 'index_id', case = 6)

Execution ID: 2eb31799-dd77-4240-b7ce-f7aa908b04e0


,test,groups,cnt_test,cnt_index_num_voie,count_test_num_voie_pct,cnt_index_type_voie,count_test_type_voie_pct,cnt_index_commune,count_test_commune_pct,cnt_index_date,count_test_date_pct,cnt_index_admin,count_test_status_admin_pct,cnt_index_siege,count_test_siege_pct,cnt_index_cp,count_test_cp_pct
0,cas_6,False,1,"149,178",36.09%,"82,285",20.17%,"1,030",0.26%,"128,325",31.07%,"170,656",40.80%,"165,770",40.06%,0,0.00%
1,cas_6,Null,1,"123,075",29.78%,"71,505",17.53%,"17,828",4.44%,"108,257",26.21%,0,0.00%,0,0.00%,0,0.00%
2,cas_6,True,1,"115,627",27.98%,"223,805",54.87%,"348,650",86.87%,"150,590",36.46%,"223,096",53.34%,"224,637",54.29%,"367,508",91.57%
3,cas_6,False,2,"13,159",3.18%,"2,618",0.64%,46,0.01%,"10,921",2.64%,"10,273",2.46%,"4,272",1.03%,0,0.00%
4,cas_6,Null,2,"4,059",0.98%,"2,131",0.52%,796,0.20%,"5,211",1.26%,0,0.00%,0,0.00%,0,0.00%
5,cas_6,True,2,"1,025",0.25%,"17,492",4.29%,"24,274",6.05%,"2,282",0.55%,"7,270",1.74%,"11,670",2.82%,"25,116",6.26%
6,cas_6,False,3,"2,620",0.63%,454,0.11%,6,0.00%,"2,374",0.57%,"1,619",0.39%,519,0.13%,0,0.00%
7,cas_6,Null,3,663,0.16%,380,0.09%,162,0.04%,699,0.17%,0,0.00%,0,0.00%,0,0.00%
8,cas_6,True,3,86,0.02%,"3,262",0.80%,"4,227",1.05%,502,0.12%,"1,704",0.41%,"2,889",0.70%,"4,395",1.10%
9,cas_6,False,4,"1,000",0.24%,180,0.04%,5,0.00%,951,0.23%,531,0.13%,111,0.03%,0,0.00%


Analyse séquence

In [55]:
compte_dup_cas(var = 'sequence_id', case = 6)

Execution ID: 84318ae4-c35b-4557-accb-45bd273f8f2e


,count_index_id,count_duplicate_index_id,percentage
0,1,"311,177",84.49%
1,2,"44,602",12.11%
2,3,"5,180",1.41%
3,4,"3,704",1.01%
4,5,672,0.18%
5,6,989,0.27%
6,7,306,0.08%
7,8,411,0.11%
8,9,146,0.04%
9,10,214,0.06%


In [56]:
generate_analytical_table_dup(var = 'sequence_id', case = 6)

Execution ID: 1ccac78c-752f-46ea-be0a-8a3aed056edb


,test,groups,cnt_test,cnt_index_num_voie,count_test_num_voie_pct,cnt_index_type_voie,count_test_type_voie_pct,cnt_index_commune,count_test_commune_pct,cnt_index_date,count_test_date_pct,cnt_index_admin,count_test_status_admin_pct,cnt_index_siege,count_test_siege_pct,cnt_index_cp,count_test_cp_pct
0,cas_6,False,1,"121,568",32.06%,"73,152",19.52%,916,0.25%,"105,676",27.63%,"144,807",37.79%,"143,792",37.55%,0,0.00%
1,cas_6,Null,1,"107,011",28.22%,"62,179",16.59%,"15,167",4.12%,"98,430",25.73%,0,0.00%,0,0.00%,0,0.00%
2,cas_6,True,1,"99,833",26.33%,"185,556",49.51%,"295,448",80.24%,"128,726",33.66%,"187,045",48.81%,"192,486",50.27%,"311,177",84.56%
3,cas_6,False,2,"21,946",5.79%,"6,292",1.68%,87,0.02%,"18,935",4.95%,"19,880",5.19%,"13,450",3.51%,0,0.00%
4,cas_6,Null,2,"10,485",2.76%,"5,896",1.57%,"1,881",0.51%,"8,876",2.32%,0,0.00%,0,0.00%,0,0.00%
5,cas_6,True,2,"8,073",2.13%,"30,326",8.09%,"42,540",11.55%,"11,997",3.14%,"21,460",5.60%,"23,265",6.08%,"44,602",12.12%
6,cas_6,False,3,"3,012",0.79%,520,0.14%,6,0.00%,"2,422",0.63%,"2,299",0.60%,"1,274",0.33%,0,0.00%
7,cas_6,Null,3,929,0.24%,528,0.14%,154,0.04%,995,0.26%,0,0.00%,0,0.00%,0,0.00%
8,cas_6,True,3,405,0.11%,"3,811",1.02%,"5,003",1.36%,792,0.21%,"2,140",0.56%,"2,997",0.78%,"5,180",1.41%
9,cas_6,False,4,"2,190",0.58%,411,0.11%,10,0.00%,"1,861",0.49%,"1,162",0.30%,368,0.10%,0,0.00%


## Cas de figure 7: Cardinality exception INPI supérieure INSEE, intersection positive 

* Definition:  L’adresse de l’INSEE contient des mots de l’adresse de l’INPI et la cardinality des mots non présents dans l’adresse de l’INPI est supérieure à la cardinality de l’adresse de l’INSEE
* Math definition: $|INPI|-|INPI \cap INSEE| > |INSEE|-|INPI \cap INSEE|$
* Règle: $|\text{insee_except}| < |\text{inpi_except}| \text{ and } \text{intersection} > 0 \rightarrow \text{cas 7}$

| list_inpi                                                                                    | list_insee                                                                   | insee_except | inpi_except                 | intersection | union_ |
|----------------------------------------------------------------------------------------------|------------------------------------------------------------------------------|--------------|-----------------------------|--------------|--------|
| [RTE, CABRIERES, D, AIGUES, CHEZ, MR, DOL, JEAN, CLAUDE, LIEUDIT, PLAN, PLUS, LOIN]          | [ROUTE, CABRIERES, D, AIGUES, CHEZ, MR, DOL, JEAN, CLAUDE, PLAN, PLUS, LOIN] | [ROUTE]      | [RTE, LIEUDIT]              | 11           | 14     |
| [ROUTE, N, ZAC, PONT, RAYONS, CC, GRAND, VAL, ILOT, B, BAT, A, LOCAL]                        | [ZONE, ZAC, PONT, RAYONS, CC, GRAND, VAL, ILOT, B, BAT, A, LOCAL]            | [ZONE]       | [ROUTE, N]                  | 11           | 14     |
| [BOULEVARD, PAUL, VALERY, BAT, B, ESC, H, APPT, C, O, MADAME, BLANDINE, BOVE]                | [BOULEVARD, PAUL, VALERY, BAT, B, ESC, H, APT, C, O, BOVE, BLANDINE]         | [APT]        | [APPT, MADAME]              | 11           | 14     |
| [RUE, JEANNE, D, ARC, A, L, ANGLE, N, ROLLON, EME, ETAGE, POLE, PRO, AGRI]                   | [RUE, JEANNE, D, ARC, A, L, ANGLE, N, ROLLON, E, ETAGE]                      | [E]          | [EME, POLE, PRO, AGRI]      | 10           | 15     |
| [CHEZ, MR, MME, DANIEL, DEZEMPTE, AVENUE, BALCONS, FRONT, MER, L, OISEAU, BLEU, BATIMENT, B] | [AVENUE, BALCONS, FRONT, MER, CHEZ, MR, MME, DANIEL, DEZEMPTE, L, OISEA]     | [OISEA]      | [OISEAU, BLEU, BATIMENT, B] | 10           | 15     |

- Nombre d'observation: 466671
    - Percentage initial: 0.04

In [57]:
cas_7 = compte_obs_cas(case= 7)

Execution ID: afaa6b6c-107c-4179-b003-138ee3e0ad1d


In [58]:
dic_['Cas de figure'].append(7)
dic_['Titre'].append('Match partiel compliqué')
dic_['Total'].append(cas_7)
dic_['Total cumulé'].append(cas_1 + cas_2 + cas_3 + cas_4 + cas_5+ cas_6 + cas_7)
dic_['pourcentage'].append(cas_7/initial_obs)
dic_['Pourcentage cumulé'].append((cas_1 + cas_2 + cas_3 + cas_4 + cas_5+ cas_6 + cas_7)/initial_obs)
dic_['Comment'].append("Match partiel compliqué")

In [59]:
generate_analytical_table(case = 7)

Execution ID: 030b22ae-8b01-493a-9705-628fe6aa9ddb


,test,groups,count_test_num_voie,count_test_num_voie_pct,count_test_type_voie,count_test_type_voie_pct,count_test_commune,count_test_commune_pct,count_test_date,count_test_date_pct,count_test_status_admin,count_test_status_admin_pct,count_test_siege,count_test_siege_pct,count_test_cp,count_test_cp_pct
0,cas_7,True,"73,185",20.26%,"230,749",63.88%,"346,229",95.84%,"124,772",34.54%,"200,897",55.61%,"223,160",61.78%,"361,242",100.00%
1,cas_7,False,"175,349",48.54%,"92,198",25.52%,421,0.12%,"151,174",41.85%,"160,345",44.39%,"138,082",38.22%,0,0.00%
2,cas_7,Null,"112,708",31.20%,"38,295",10.60%,"14,592",4.04%,"85,296",23.61%,0,0.00%,0,0.00%,0,0.00%


Analyse index

In [60]:
compte_dup_cas(var = 'index_id', case = 7)

Execution ID: 67d9de4d-22b2-4ede-9703-0d96c95c71b5


,count_index_id,count_duplicate_index_id,percentage
0,1,"296,702",92.97%
1,2,"16,116",5.05%
2,3,"3,087",0.97%
3,4,"1,193",0.37%
4,5,615,0.19%
5,6,376,0.12%
6,7,264,0.08%
7,8,167,0.05%
8,9,84,0.03%
9,10,78,0.02%


In [61]:
generate_analytical_table_dup(var = 'index_id', case = 7)

Execution ID: 2e794c4a-b108-48e8-88ae-8b7ff0c4643c


,test,groups,cnt_test,cnt_index_num_voie,count_test_num_voie_pct,cnt_index_type_voie,count_test_type_voie_pct,cnt_index_commune,count_test_commune_pct,cnt_index_date,count_test_date_pct,cnt_index_admin,count_test_status_admin_pct,cnt_index_siege,count_test_siege_pct,cnt_index_cp,count_test_cp_pct
0,cas_7,False,1,"138,999",42.86%,"84,603",26.26%,380,0.12%,"114,508",35.24%,"137,574",41.85%,"130,668",40.30%,0,0.00%
1,cas_7,Null,1,"94,686",29.20%,"30,250",9.39%,"12,763",4.00%,"76,615",23.58%,0,0.00%,0,0.00%,0,0.00%
2,cas_7,True,1,"71,630",22.09%,"186,222",57.81%,"283,559",88.90%,"115,524",35.55%,"173,412",52.75%,"175,751",54.20%,"296,702",93.02%
3,cas_7,False,2,"9,767",3.01%,"2,214",0.69%,11,0.00%,"8,164",2.51%,"6,949",2.11%,"2,853",0.88%,0,0.00%
4,cas_7,Null,2,"3,233",1.00%,"1,197",0.37%,529,0.17%,"2,922",0.90%,0,0.00%,0,0.00%,0,0.00%
5,cas_7,True,2,578,0.18%,"11,779",3.66%,"15,576",4.88%,"1,694",0.52%,"5,838",1.78%,"9,376",2.89%,"16,116",5.05%
6,cas_7,False,3,"2,046",0.63%,410,0.13%,1,0.00%,"1,909",0.59%,"1,142",0.35%,322,0.10%,0,0.00%
7,cas_7,Null,3,599,0.18%,256,0.08%,94,0.03%,385,0.12%,0,0.00%,0,0.00%,0,0.00%
8,cas_7,True,3,62,0.02%,"2,337",0.73%,"2,992",0.94%,412,0.13%,"1,357",0.41%,"2,369",0.73%,"3,087",0.97%
9,cas_7,False,4,747,0.23%,156,0.05%,0,0.00%,757,0.23%,357,0.11%,73,0.02%,0,0.00%


Analyse séquence

In [62]:
compte_dup_cas(var = 'sequence_id', case = 7)

Execution ID: 054e5c05-c3b0-4289-b684-f47a9acc2f45


,count_index_id,count_duplicate_index_id,percentage
0,1,"246,372",84.91%
1,2,"34,615",11.93%
2,3,"3,917",1.35%
3,4,"2,719",0.94%
4,5,517,0.18%
5,6,743,0.26%
6,7,228,0.08%
7,8,292,0.10%
8,9,76,0.03%
9,10,131,0.05%


In [63]:
generate_analytical_table_dup(var = 'sequence_id', case = 7)

Execution ID: adb409c2-9ca2-4ae0-b1a2-e659589b7149


,test,groups,cnt_test,cnt_index_num_voie,count_test_num_voie_pct,cnt_index_type_voie,count_test_type_voie_pct,cnt_index_commune,count_test_commune_pct,cnt_index_date,count_test_date_pct,cnt_index_admin,count_test_status_admin_pct,cnt_index_siege,count_test_siege_pct,cnt_index_cp,count_test_cp_pct
0,cas_7,False,1,"111,354",37.76%,"73,276",24.99%,327,0.11%,"93,130",31.07%,"114,329",38.30%,"112,412",37.71%,0,0.00%
1,cas_7,Null,1,"81,243",27.55%,"26,004",8.87%,"10,657",3.67%,"69,729",23.26%,0,0.00%,0,0.00%,0,0.00%
2,cas_7,True,1,"60,843",20.63%,"151,304",51.60%,"235,636",81.22%,"98,541",32.88%,"142,809",47.83%,"147,889",49.61%,"246,372",84.97%
3,cas_7,False,2,"19,433",6.59%,"6,829",2.33%,34,0.01%,"16,189",5.40%,"15,799",5.29%,"10,514",3.53%,0,0.00%
4,cas_7,Null,2,"8,562",2.90%,"2,838",0.97%,"1,399",0.48%,"5,501",1.84%,0,0.00%,0,0.00%,0,0.00%
5,cas_7,True,2,"5,386",1.83%,"24,334",8.30%,"33,128",11.42%,"9,264",3.09%,"17,942",6.01%,"19,496",6.54%,"34,615",11.94%
6,cas_7,False,3,"2,515",0.85%,600,0.20%,2,0.00%,"1,937",0.65%,"1,715",0.57%,959,0.32%,0,0.00%
7,cas_7,Null,3,823,0.28%,301,0.10%,115,0.04%,641,0.21%,0,0.00%,0,0.00%,0,0.00%
8,cas_7,True,3,305,0.10%,"2,933",1.00%,"3,783",1.30%,647,0.22%,"1,830",0.61%,"2,467",0.83%,"3,917",1.35%
9,cas_7,False,4,"1,682",0.57%,392,0.13%,1,0.00%,"1,484",0.50%,861,0.29%,257,0.09%,0,0.00%


## Recapitulatif cas

In [64]:
(pd.DataFrame(dic_)
 .style
 .format("{:,.0f}", subset =  ['Total',
                                            'Total cumulé'])
              .format("{:.2%}", subset =  ['pourcentage',
                                           'Pourcentage cumulé'])
              .bar(subset= ['Total',
                                            'Total cumulé'],
                   color='#d65f5f')
)

,Cas de figure,Titre,Total,Total cumulé,pourcentage,Pourcentage cumulé,Comment
0,1,similarité parfaite,"7,775,392","7,775,392",67.03%,67.03%,Match parfait
1,2,Exclusion parfaite,"974,444","8,749,836",8.40%,75.43%,Exclusion parfaite
2,3,Match partiel parfait,"407,404","9,157,240",3.51%,78.94%,Match partiel parfait
3,4,Match partiel parfait,"558,992","9,716,232",4.82%,83.76%,Match partiel parfait
4,5,Match partiel compliqué,"1,056,406","10,772,638",9.11%,92.86%,Match partiel compliqué
5,6,Match partiel compliqué,"361,242","11,133,880",3.11%,95.98%,Match partiel compliqué
6,7,Match partiel compliqué,"466,671","11,600,551",4.02%,100.00%,Match partiel compliqué


In [66]:
#print(pd.DataFrame(dic_).set_index('Cas de figure').to_markdown())

## Resume match

Le tableau ci dessous résume les matches si on n'utilise qu'un seul des tests sur les index uniques. Par exemple, il y a 7,443,607 lignes uniques pour le cas numéro 1. Parmi ses 7,443,607, il y a 6,443,723 lignes uniques qui ont passé le test du numéro de voie. L'ensemble des lignes uniques qui ont passé le test du numéro de voie représente environ 70% des lignes de l'INPI

| Test              | Cas 1                           | Cas 2             | Cas 3              | Cas 4             | Cas 5                  | Cas 6             | Cas 7                | SUM             | pourcentage INPI |
|-------------------|---------------------------------|-------------------|--------------------|-------------------|------------------------|-------------------|----------------------|-----------------|------------------|
| Total analyse     |                7 775 392        |          974 444  |          407 404   |          558 992  |         1 056 406      |         361 242   |         466 671      |    11 600 551   |                  |
| Index unique      |                7 443 607        |                   |          387 589   |          525 832  |              943 789   |         367 508   |         296 702      |      9 965 027  | 89,47%           |
|                   |      test count (1) + True      |                   |                    |                   |                        |                   |                      |                 |                  |
| Test numéro voie  |                6 443 723        |                   |          254 858   |          320 206  |              624 127   |         115 627   |               7 163  |      7 765 704  | 69,72%           |
| Test type voie    |                6 676 760        |                   |          275 559   |          361 456  |              628 397   |         223 805   |         186 222      |      8 352 199  | 74,99%           |
| Test date         |                    3 757   064  |                   |          179 688   |          253 205  |                44 084  |           15 059  |         115 524      |      4 364 624  | 39,19%           |
| Test status admin |                    6 246   835  |                   |            31 054  |          418 014  |              679 414   |         223 096   |         173 412      |      7 771 825  | 69,78%           |
| Test siege        |                    4 500   976  |                   |          234 578   |          331 856  |              561 655   |         224 637   |           175 751    |      6 029 453  | 54,13%           |

# Generate report

In [67]:
import os, time, shutil, urllib, ipykernel, json
from pathlib import Path
from notebook import notebookapp

In [68]:
def create_report(extension = "html"):
    """
    Create a report from the current notebook and save it in the 
    Report folder (Parent-> child directory)
    
    1. Exctract the current notbook name
    2. Convert the Notebook 
    3. Move the newly created report
    
    Args:
    extension: string. Can be "html", "pdf", "md"
    
    
    """
    
    ### Get notebook name
    connection_file = os.path.basename(ipykernel.get_connection_file())
    kernel_id = connection_file.split('-', 1)[0].split('.')[0]

    for srv in notebookapp.list_running_servers():
        try:
            if srv['token']=='' and not srv['password']:  
                req = urllib.request.urlopen(srv['url']+'api/sessions')
            else:
                req = urllib.request.urlopen(srv['url']+ \
                                             'api/sessions?token=' + \
                                             srv['token'])
            sessions = json.load(req)
            notebookname = sessions[0]['name']
        except:
            pass  
    
    sep = '.'
    path = os.getcwd()
    #parent_path = str(Path(path).parent)
    
    ### Path report
    #path_report = "{}/Reports".format(parent_path)
    #path_report = "{}/Reports".format(path)
    
    ### Path destination
    name_no_extension = notebookname.split(sep, 1)[0]
    source_to_move = name_no_extension +'.{}'.format(extension)
    dest = os.path.join(path,'Reports', source_to_move)
    
    ### Generate notebook
    os.system('jupyter nbconvert --no-input --to {} {}'.format(
    extension,notebookname))
    
    ### Move notebook to report folder
    #time.sleep(5)
    shutil.move(source_to_move, dest)
    print("Report Available at this adress:\n {}".format(dest))

In [69]:
create_report(extension = "html")

Report Available at this adress:
 C:\Users\PERNETTH\Documents\Projects\InseeInpi_matching\Notebooks_matching\Data_preprocessed\programme_matching\02_siretisation\Reports\06_analyse_pre_siretisation_v3.html
